In [42]:
# Configuration

import logging
import sys
import os
import pandas as pd
from datetime import datetime
from pathlib import Path


rightnow = datetime.now().strftime("%Y-%m-%d-%H-%M-%S")
LINEAGE_LEVEL = "family"
HMM_FILE = "ascogs_escrt_system_20260116_121403.tsv"
FLANK_K = 0               # number of 'other' tokens allowed at each flank

NOTEBOOK_PATH = Path.cwd() / "ESCRT_system_synteny.ipynb"

WINDOW = 5
CORE_TARGETS = ("vps", "escrt", "katanin", "eap", "flad")

MAIN_OUTDIR = os.path.join(os.getcwd(), f"ESCRT_synteny_pipeline_output_{LINEAGE_LEVEL}_{rightnow}")
os.makedirs(MAIN_OUTDIR, exist_ok=True)

def setup_logging(log_file=None):
    """
    Configure logging for the pipeline.
    Logs to stdout and optionally to a file.
    """
    handlers = [logging.StreamHandler(sys.stdout)]
    if log_file:
        handlers.append(logging.FileHandler(log_file))

    logging.basicConfig(
        level=logging.INFO,
        format="%(asctime)s [%(levelname)s] %(message)s",
        handlers=handlers,
        force=True  # Force reconfiguration if already set
    )


STEP = 0

setup_logging(os.path.join(MAIN_OUTDIR, f"ESCRT_synteny_pipeline{rightnow}.log"))


In [43]:
# Merge HMM hits with protein metadata


# ----------------------------
# Input files
# ----------------------------
hits_file = "/home/anirudh/synteny/hmms/ESCRT_results/hits/ESCRT_hits_final.csv"
proteins_file = "/home/anirudh/synteny/proteins_genomes_cp90_con5.csv"

# ----------------------------
# Output file naming
# ----------------------------
hits_file_name = os.path.basename(hits_file).split(".")[0]
STEP += 1
merge_file_name = os.path.join(MAIN_OUTDIR, f"[STEP:{STEP}]{hits_file_name}_merged_{rightnow}.csv")

logging.info("HMM hits file: %s", hits_file)
logging.info("Protein metadata file: %s", proteins_file)
logging.info("Merged output file: %s", merge_file_name)

# ----------------------------
# Load HMM hits
# ----------------------------
try:
    hits = pd.read_csv(hits_file)
except Exception as e:
    logging.error("Failed to load HMM hits file: %s", e)
    raise

hits.columns = hits.columns.str.strip()
logging.info("HMM hits columns: %s", list(hits.columns))
logging.info("HMM hits rows: %d", len(hits))

# ----------------------------
# Load protein metadata
# ----------------------------
try:
    proteins = pd.read_csv(proteins_file)
except Exception as e:
    logging.error("Failed to load protein metadata file: %s", e)
    raise

proteins.columns = proteins.columns.str.strip()
logging.info("Protein metadata columns: %s", list(proteins.columns))
logging.info("Protein metadata rows: %d", len(proteins))

# ----------------------------
# Sanity checks before merge
# ----------------------------
required_hits_cols = {"target"}
required_prot_cols = {"cds_id"}

missing_hits = required_hits_cols - set(hits.columns)
missing_prots = required_prot_cols - set(proteins.columns)

if missing_hits:
    logging.error("Missing required columns in HMM hits file: %s", missing_hits)
    raise ValueError(f"Missing columns in hits file: {missing_hits}")

if missing_prots:
    logging.error("Missing required columns in protein metadata file: %s", missing_prots)
    raise ValueError(f"Missing columns in proteins file: {missing_prots}")

logging.info("Input sanity checks passed")

# ----------------------------
# Merge HMM hits with protein metadata
# ----------------------------
logging.info("Merging HMM hits with protein metadata")
logging.info("Rows before merge: hits=%d, proteins=%d", len(hits), len(proteins))

merged = pd.merge(
    hits,
    proteins,
    left_on="target",
    right_on="cds_id",
    how="inner"
)

logging.info("Rows after merge: %d", len(merged))
logging.info("Unique targets merged: %d", merged["target"].nunique())

# ----------------------------
# Write output
# ----------------------------
try:
    
    merged.to_csv(merge_file_name, index=False)
except Exception as e:
    logging.error("Failed to write merged output file %s: %s", merge_file_name, e)
    raise

logging.info("Merged output written to: %s", merge_file_name)


2026-01-21 11:08:12,880 [INFO] HMM hits file: /home/anirudh/synteny/hmms/ESCRT_results/hits/ESCRT_hits_final.csv
2026-01-21 11:08:12,881 [INFO] Protein metadata file: /home/anirudh/synteny/proteins_genomes_cp90_con5.csv
2026-01-21 11:08:12,881 [INFO] Merged output file: /home/anirudh/synteny/hmms/ESCRT_synteny_pipeline_output_family_2026-01-21-11-08-12/[STEP:1]ESCRT_hits_final_merged_2026-01-21-11-08-12.csv
2026-01-21 11:08:12,905 [INFO] HMM hits columns: ['target', 'tacc', 'tlen', 'query', 'qacc', 'qlen', 'full_evalue', 'full_score', 'full_bias', 'dom_idx', 'dom_count', 'c_evalue', 'i_evalue', 'dom_score', 'dom_bias', 'hmm_from', 'hmm_to', 'ali_from', 'ali_to', 'env_from', 'env_to', 'acc', 'description', 'source_file']
2026-01-21 11:08:12,906 [INFO] HMM hits rows: 24597
2026-01-21 11:08:16,911 [INFO] Protein metadata columns: ['Name', 'Completeness', 'Contamination', 'Completeness_Model_Used', 'Translation_Table_Used', 'Coding_Density', 'Contig_N50', 'Average_Gene_Length', 'Genome_Siz

Merge with the ascogs definition

In [44]:
# Merge HMM hits with AsCOG annotations

# ----------------------------
# Load AsCOG annotation table
# ----------------------------
ascogs_tsv = f"/home/anirudh/synteny/hmms/Sources/{HMM_FILE}"
logging.info("Loading AsCOGs from: %s", ascogs_tsv)

try:
    ascogs = pd.read_csv(ascogs_tsv, sep="\t")
except Exception as e:
    logging.error("Failed to load AsCOGs file: %s", e)
    raise

ascogs.columns = ascogs.columns.str.strip()

logging.info("AsCOGs columns: %s", list(ascogs.columns))
logging.info("AsCOGs rows: %d", len(ascogs))

# ----------------------------
# Sanity checks
# ----------------------------
required_cols = {"ascog_id"}
missing = required_cols - set(ascogs.columns)

if missing:
    logging.error("Missing required columns in AsCOGs file: %s", missing)
    raise ValueError(f"Missing required columns: {missing}")

logging.info("AsCOGs sanity check passed")

# ----------------------------
# Merge HMM hits with AsCOGs
# ----------------------------
logging.info("Merging HMM hits with AsCOG annotations")
logging.info("HMM hits rows before merge: %d", len(merged))

merged_ascogs = pd.merge(
    merged,
    ascogs,
    left_on="query",
    right_on="ascog_id",
    how="left"
)

logging.info("Rows after AsCOG merge: %d", len(merged_ascogs))
logging.info("Unique AsCOGs matched: %d", merged_ascogs["ascog_id"].nunique())
logging.info("Unique proteins matched: %d", merged_ascogs["target"].nunique())

# ----------------------------
# Write output
# ----------------------------
STEP += 1
final_output_file = os.path.join(MAIN_OUTDIR, f"[STEP:{STEP}]ESCRT_hits_final_merged_{rightnow}_ascog.csv")

merged_ascogs = merged_ascogs[["target", "tacc", "tlen", "query", "qacc", "qlen", "c_evalue", "i_evalue", "dom_score",  "hmm_from", "hmm_to", "ali_from", "ali_to", "env_from", "env_to", "acc", "Name", "Completeness", "Contamination", "Contig_N50", "Total_Contigs", "organism_name", "cds_id", "header", "gene", "description_y"]]

try:
    merged_ascogs.to_csv(final_output_file, index=False)
except Exception as e:
    logging.error("Failed to write output file %s: %s", final_output_file, e)
    raise

logging.info("Final merged file written to: %s", final_output_file)


2026-01-21 11:08:17,290 [INFO] Loading AsCOGs from: /home/anirudh/synteny/hmms/Sources/ascogs_escrt_system_20260116_121403.tsv
2026-01-21 11:08:17,292 [INFO] AsCOGs columns: ['ascog_id', 'arcog_id', 'category', 'gene', 'description']
2026-01-21 11:08:17,293 [INFO] AsCOGs rows: 89
2026-01-21 11:08:17,294 [INFO] AsCOGs sanity check passed
2026-01-21 11:08:17,294 [INFO] Merging HMM hits with AsCOG annotations
2026-01-21 11:08:17,295 [INFO] HMM hits rows before merge: 24597
2026-01-21 11:08:17,305 [INFO] Rows after AsCOG merge: 24597
2026-01-21 11:08:17,306 [INFO] Unique AsCOGs matched: 88
2026-01-21 11:08:17,308 [INFO] Unique proteins matched: 7285
2026-01-21 11:08:17,434 [INFO] Final merged file written to: /home/anirudh/synteny/hmms/ESCRT_synteny_pipeline_output_family_2026-01-21-11-08-12/[STEP:2]ESCRT_hits_final_merged_2026-01-21-11-08-12_ascog.csv


In [45]:
# Select best hits based on e-value and coverage

merged_ascogs["coverage"] = (
    (merged_ascogs["hmm_to"] - merged_ascogs["hmm_from"] + 1)
    / merged_ascogs["qlen"]
)

merged_ascogs = (
    merged_ascogs
    .sort_values(
        by=["target", "query", "i_evalue", "coverage", "dom_score"],
        ascending=[True, True, True, False, False]
    )
)
best_hits = (
    merged_ascogs
    .drop_duplicates(subset=["target", "query"], keep="first")
)

best_hits = best_hits[(best_hits["i_evalue"] <= 1e-5) & (best_hits["coverage"] >= 0.65)]

best_hits = best_hits[["target","query","gene","dom_score","i_evalue","coverage","description_y","tacc","tlen","qacc","qlen","c_evalue","hmm_from","hmm_to","ali_from","ali_to","env_from","env_to","acc","Name","Completeness","Contamination","Contig_N50","Total_Contigs","organism_name"]]

STEP += 1
best_hits_file = os.path.join(MAIN_OUTDIR, f"[STEP:{STEP}]ESCRT_hits_final_merged_{rightnow}_ascog_best_hits.csv")

best_hits.to_csv(best_hits_file, index=False)
logging.info("Best hits file written to: %s", best_hits_file)


2026-01-21 11:08:17,494 [INFO] Best hits file written to: /home/anirudh/synteny/hmms/ESCRT_synteny_pipeline_output_family_2026-01-21-11-08-12/[STEP:3]ESCRT_hits_final_merged_2026-01-21-11-08-12_ascog_best_hits.csv


In [46]:
# Compute protein counts with AsCOG hits


protein_counts = (
    best_hits
    .dropna(subset=["gene", "query"])
    .assign(
        gene=lambda df: df["gene"].str.strip(),
        ascog_id=lambda df: df["query"].str.strip()
    )
    .groupby("target")
    .agg(
        ascog_hits=("gene", lambda x: list(x.unique())),
        ascog_ids=("query", lambda x: list(x.unique()))
    )
    .reset_index()
    .assign(
        ascog_count=lambda df: df["ascog_hits"].apply(len)
    )
    .sort_values(by = ["ascog_count"], ascending=[False])
    .assign(
        arch_str=lambda d: d["ascog_hits"].apply(lambda x: "+".join(sorted(x)))
    )
    .sort_values(
        by=["ascog_count", "arch_str"],
        ascending=[False, True]
    )
)

logging.info("Computed protein counts with AsCOG hits")

STEP += 1
protein_counts_file = os.path.join(MAIN_OUTDIR, f"[STEP:{STEP}]protein_ascog_counts_{rightnow}.csv")
try:
    protein_counts.to_csv(protein_counts_file, index=False)
except Exception as e:
    logging.error("Failed to write protein counts file %s: %s", protein_counts_file, e)
    raise ValueError(f"Failed to write protein counts file: {e}")

2026-01-21 11:08:17,576 [INFO] Computed protein counts with AsCOG hits


In [47]:
# Infer architectures based on rules

# Order matters: first match wins
ARCH_RULES = [
    # Vps23 canonical
    ({"CC-Vps23", "PH-Vps23", "Vps23"}, "CC-PH-Vps23"),
    ({"CC-Vps23", "PH-Vps23"}, "CC-PH-Vps23"),

    # E2–Vps23 fusion
    ({"E2-Vps23", "E2", "Vps23"}, "E2-Vps23"),
    ({"E2-Vps23", "E2"}, "E2-Vps23"),
    ({"E2-Vps23", "Vps23"}, "E2-Vps23"),

    # ESCRT-I–linked E2
    ({"Vps28", "E2-Vps23", "E2"}, "E2-ESCRT-I"),

    # ESCRT-II Vps22-Vps36 fusion
    ({"CC-Vps23", "EAP30"}, "Vps23-EAP30"),

    # MPN-E3_doms
    ({"MPN", "E3-dom"}, "MPN-E3-dom"),
]



import pandas as pd

def infer_architecture(group: pd.DataFrame) -> pd.Series:
    """
    Infer a single architecture for one protein (target)
    using explicit rules, else fallback to best i-Evalue hit.
    """

    # Clean gene names
    genes = set(
        group["gene"]
        .dropna()
        .astype(str)
        .str.strip()
    )

    # 1. Apply rule-based architecture inference
    for rule_genes, architecture in ARCH_RULES:
        if rule_genes.issubset(genes):
            return pd.Series({
                "architecture": architecture,
                "architecture_method": "rule",
                "architecture_components": ",".join(sorted(rule_genes))
            })

    # 2. Fallback: best single hit by independent E-value
    best_hit = (
        group
        .dropna(subset=["i_evalue"])
        .sort_values("i_evalue", ascending=True)
        .iloc[0]
    )

    return pd.Series({
        "architecture": best_hit["gene"],
        "architecture_method": "best_i_evalue",
        "architecture_components": best_hit["gene"]
    })



architecture_df = (
    best_hits
    .dropna(subset=["target", "gene", "i_evalue"])
    .groupby("target", group_keys=False)
    .apply(infer_architecture)
    .reset_index()
)



df_with_arch = best_hits.merge(
    architecture_df,
    on="target",
    how="left"
)


df_with_arch = df_with_arch[["target","Name","Completeness","Contamination","Contig_N50","Total_Contigs","organism_name","architecture","architecture_method","architecture_components"]]

df_with_arch.drop_duplicates(inplace=True)
logging.info("Inferred architectures for proteins")

STEP += 1
df_with_arch_file = os.path.join(MAIN_OUTDIR, f"[STEP:{STEP}]ESCRT_hits_with_architectures_{rightnow}.csv")
try:
    df_with_arch.to_csv(df_with_arch_file, index=False)
except Exception as e: 
    logging.error("Failed to write architecture output file %s: %s", df_with_arch_file, e)
    raise ValueError(f"Failed to write architecture output file: {e}")

2026-01-21 11:08:19,662 [INFO] Inferred architectures for proteins


/tmp/ipykernel_930/1008619236.py:71: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(infer_architecture)


In [48]:

# # ------------------------------------------------------------
# # Parse Prodigal GFF and assign gene order PER CONTIG
# # ------------------------------------------------------------
# def parse_prodigal_gff(gff_path, genome_id):
#     """
#     Parse a Prodigal GFF3 file and extract CDS gene order per contig.

#     Gene order is defined by appearance order in the GFF,
#     which corresponds to genomic order for Prodigal output.
    
#     Args:
#         gff_path: Path to the GFF file
#         genome_id: Identifier for the genome
        
#     Returns:
#         DataFrame with columns: genome_id, contig, gene_index, 
#                                 protein_id, start, end, strand
#     """

#     rows = []
#     gene_counter = {}  # separate gene index per contig

#     logging.info(f"Parsing GFF: {gff_path.name}")

#     with open(gff_path) as f:
#         for line in f:
#             # Skip comment lines
#             if line.startswith("#"):
#                 continue

#             parts = line.rstrip().split("\t")
#             if len(parts) != 9:
#                 continue

#             contig, source, feature, start, end, score, strand, phase, attrs = parts

#             # Only process CDS features
#             if feature != "CDS":
#                 continue

#             # Parse attributes to extract protein ID
#             attr_dict = {}
#             for item in attrs.split(";"):
#                 if "=" in item:
#                     k, v = item.split("=", 1)
#                     attr_dict[k] = v

#             protein_id = attr_dict.get("ID")
#             if protein_id is None:
#                 continue

#             # Increment gene index PER CONTIG (not genome-wide)
#             gene_counter.setdefault(contig, 0)
#             gene_counter[contig] += 1

#             rows.append({
#                 "genome_id": genome_id,
#                 "contig": contig,
#                 "gene_index": gene_counter[contig],
#                 "protein_id": protein_id,
#                 "start": int(start),
#                 "end": int(end),
#                 "strand": strand
#             })

#     logging.info(f"  → Parsed {len(rows)} CDS features across {len(gene_counter)} contigs")
#     return pd.DataFrame(rows)


    

# # ------------------------------------------------------------
# # Load only the GFFs required by the hits table
# # ------------------------------------------------------------
# def load_gffs_from_hits(hits_df, gff_dir):
#     """
#     Parse GFF files only for genomes present in the hits dataframe.
#     GFF filenames are inferred as: <genome_file>_genomic/<genome_file>_genomic.gff
    
#     Args:
#         hits_df: DataFrame containing hits with 'genome_file' column
#         gff_dir: Directory containing GFF files
        
#     Returns:
#         Combined DataFrame of all parsed GFF files
#     """

#     all_gff_rows = []
    
#     unique_genomes = hits_df["Name"].unique()
#     logging.info(f"Loading GFFs for {len(unique_genomes)} unique genomes")

#     for genome_id in unique_genomes:
#         gff_path = gff_dir / f"{genome_id}_genomic" / f"{genome_id}_genomic.gff"

#         if not gff_path.exists():
#             logging.warning(f"Missing GFF: {gff_path}")
#             continue

#         gff_df = parse_prodigal_gff(gff_path, genome_id)
#         all_gff_rows.append(gff_df)

#     if not all_gff_rows:
#         logging.error("No GFF files were successfully loaded.")
#         sys.exit(1)

#     combined_gff = pd.concat(all_gff_rows, ignore_index=True)
#     logging.info(f"Total CDS features loaded: {len(combined_gff)}")
    
#     return combined_gff





# def extract_neighborhoods(anchor_df, gff_df, window=5):
#     """
#     Extract ±window gene neighborhoods around ESCRT-related anchor genes only.
#     Anchors are filtered by architecture name containing:
#     vps, escrt, or katanin (case-insensitive).
#     """

#     blocks = []

#     # keywords defining ESCRT relevance
#     keywords = ("vps", "escrt", "katanin", "eap", "flad", "zn-ph")

#     logging.info(f"Starting neighborhood extraction (±{window} genes)")
#     logging.info(f"Initial anchors: {len(anchor_df)}")

#     # Filter anchors by architecture name
#     filtered_anchors = anchor_df[
#         anchor_df["architecture"]
#         .astype(str)
#         .str.lower()
#         .str.contains("|".join(keywords))
#     ]

#     filtered_anchors.to_csv(os.path.join(MAIN_OUTDIR, f"[STEP:{STEP}.5]filtered_anchors_ESCRT_{rightnow}.csv"), index=False)

#     logging.info(
#         f"Anchors after ESCRT filter: {len(filtered_anchors)} "
#         f"(skipped {len(anchor_df) - len(filtered_anchors)})"
#     )

#     for _, row in filtered_anchors.iterrows():
#         genome = row["genome_id"]
#         contig = row["contig"]
#         center = row["gene_index"]

#         if blocks and center in blocks[-1]['gene_index']:
#             continue

#         block = gff_df[
#             (gff_df["genome_id"] == genome) &
#             (gff_df["contig"] == contig) &
#             (gff_df["gene_index"].between(center - window, center + window))
#         ].copy()

#         if block.empty:
#             continue

#         block["center_protein"] = row["protein_id"]
#         block["center_gene"] = row["architecture"]
#         block["relative_pos"] = block["gene_index"] - center

#         blocks.append(block)

#     if not blocks:
#         logging.error("No neighborhood blocks extracted after ESCRT filtering!")
#         sys.exit(1)

#     combined_neighborhoods = pd.concat(blocks, ignore_index=True)

#     logging.info(
#         f"Extracted {len(combined_neighborhoods)} genes "
#         f"from {len(blocks)} ESCRT-related neighborhoods"
#     )

#     return combined_neighborhoods




# def build_windows(anchor_df, window):
#     """
#     Build genomic windows (start, end) around anchor genes.
#     """
#     windows = []

#     for _, row in anchor_df.iterrows():
#         windows.append({
#             "genome_id": row["genome_id"],
#             "contig": row["contig"],
#             "winstart": row["gene_index"] - window,
#             "winend": row["gene_index"] + window,
#             "wincenter": row["gene_index"],
#             "protein_id": row["protein_id"],
#             "architecture": row["architecture"],
#         })

#     return pd.DataFrame(windows)


# def merge_windows(windows_df):
#     """
#     Merge overlapping windows per genome/contig.
#     """
#     merged = []

#     for (genome, contig), grp in windows_df.groupby(["genome_id", "contig"]):
#         grp = grp.sort_values("winstart")

#         cur_start = None
#         cur_end = None
#         cur_centers = []

#         for _, row in grp.iterrows():
#             if cur_start is None:
#                 cur_start = row["winstart"]
#                 cur_end = row["winend"]
#                 cur_centers = [row]
#                 continue

#             if row["winstart"] <= cur_end + 1:
#                 # overlap → extend
#                 cur_end = max(cur_end, row["winend"])
#                 cur_centers.append(row)
#             else:
#                 merged.append({
#                     "genome_id": genome,
#                     "contig": contig,
#                     "winstart": cur_start,
#                     "winend": cur_end,
#                     "anchors": cur_centers
#                 })
#                 cur_start = row["winstart"]
#                 cur_end = row["winend"]
#                 cur_centers = [row]

#         merged.append({
#             "genome_id": genome,
#             "contig": contig,
#             "winstart": cur_start,
#             "winend": cur_end,
#             "anchors": cur_centers
#         })

#     return merged



# def extract_merged_neighborhoods(merged_windows, gff_df):
#     blocks = []

#     for win in merged_windows:
#         block = gff_df[
#             (gff_df["genome_id"] == win["genome_id"]) &
#             (gff_df["contig"] == win["contig"]) &
#             (gff_df["gene_index"].between(win["winstart"], win["winend"]))
#         ].copy()

#         if block.empty:
#             continue

#         # pick a representative anchor (central-most)
#         centers = [a["wincenter"] for a in win["anchors"]]
#         rep_center = min(centers, key=lambda x: abs(x - (win["winstart"] + win["winend"]) / 2))

#         block["relative_pos"] = block["gene_index"] - rep_center
#         block["center_protein"] = win["anchors"][0]["protein_id"]
#         block["center_gene"] = win["anchors"][0]["architecture"]

#         blocks.append(block)

#     return pd.concat(blocks, ignore_index=True)



# def extract_neighborhoods(anchor_df, gff_df, window=5):

#     keywords = ("vps", "escrt", "katanin", "eap", "flad", "zn-ph")

#     logging.info(f"Starting neighborhood extraction (±{window} genes)")
#     logging.info(f"Initial anchors: {len(anchor_df)}")

#     filtered_anchors = anchor_df[
#         anchor_df["architecture"]
#         .astype(str)
#         .str.lower()
#         .str.contains("|".join(keywords))
#     ]

#     logging.info(f"Anchors after ESCRT filter: {len(filtered_anchors)}")

#     if filtered_anchors.empty:
#         logging.error("No ESCRT-related anchors found")
#         sys.exit(1)

#     windows_df = build_windows(filtered_anchors, window)
#     merged_windows = merge_windows(windows_df)

#     logging.info(
#         f"Merged {len(windows_df)} anchor windows into "
#         f"{len(merged_windows)} non-overlapping regions"
#     )

#     combined_neighborhoods = extract_merged_neighborhoods(
#         merged_windows, gff_df
#     )

#     logging.info(
#         f"Extracted {len(combined_neighborhoods)} genes "
#         f"from {len(merged_windows)} merged neighborhoods"
#     )

#     return combined_neighborhoods


# # ------------------------------------------------------------
# # Load GTDB taxonomy
# # ------------------------------------------------------------
# def load_gtdb_taxonomy(gtdb_tsv):
#     """
#     Load GTDB taxonomy file and split into rank-specific columns.
    
#     GTDB format: genome_id\td__Domain;p__Phylum;c__Class;o__Order;f__Family;g__Genus;s__Species
    
#     Args:
#         gtdb_tsv: Path to GTDB taxonomy TSV file
        
#     Returns:
#         DataFrame with columns: genome_id, domain, phylum, class, order, family, genus, species
#     """
#     logging.info(f"Loading GTDB taxonomy from: {gtdb_tsv}")
    
#     # Read the taxonomy file
#     tax_df = pd.read_csv(
#         gtdb_tsv,
#         sep="\t",
#         header=None,
#         names=["genome_id", "taxonomy"],
#         dtype=str
#     )
    
#     tax_df["genome_id_base"] = tax_df["genome_id"].str.split("_", n=1).str[1]
#     logging.info(f"Loaded taxonomy for {len(tax_df)} genomes")
#     logging.info(f"Sample taxonomy entry: {tax_df['taxonomy'].iloc[0]}")
    
#     # Split taxonomy string by semicolons
#     tax_split = tax_df["taxonomy"].str.split(";", expand=True)
#     logging.info(f"Taxonomy split into {tax_split.shape[1]} columns")
    
#     # Map column indices to taxonomic ranks
#     rank_map = {
#         0: "domain",
#         1: "phylum",
#         2: "class",
#         3: "order",
#         4: "family",
#         5: "genus",
#         6: "species"
#     }
    
#     # Extract each rank and remove the prefix (e.g., "d__", "p__")
#     for idx, rank in rank_map.items():
#         if idx < tax_split.shape[1]:
#             # Remove the rank prefix using regex (e.g., "p__" from "p__Crenarchaeota")
#             tax_df[rank] = tax_split[idx].str.replace(r"^[a-z]__", "", regex=True)
#             logging.info(f"Extracted {rank}: {tax_df[rank].notna().sum()} non-null values")
#         else:
#             tax_df[rank] = pd.NA
#             logging.warning(f"Column {idx} for rank '{rank}' not found in taxonomy data")
    
#     # Drop the original concatenated taxonomy string
#     tax_df.drop(columns=["taxonomy"], inplace=True)
    
#     # Log sample of parsed taxonomy
#     logging.info(f"Sample parsed taxonomy:")
#     logging.info(tax_df[["genome_id", "domain", "phylum", "class"]].head().to_string())
    
#     return tax_df


# # ------------------------------------------------------------
# # Merge taxonomy into neighborhood data
# # ------------------------------------------------------------
# def merge_taxonomy(escrt_csv, gtdb_tsv, out_csv):
#     """
#     Merge GTDB taxonomy into ESCRT neighborhood dataframe.
    
#     Args:
#         escrt_csv: Path to ESCRT neighborhoods CSV
#         gtdb_tsv: Path to GTDB taxonomy TSV
#         out_csv: Output path for merged CSV
        
#     Returns:
#         Merged DataFrame with taxonomy columns added
#     """
#     logging.info("=" * 70)
#     logging.info("STARTING TAXONOMY MERGE")
#     logging.info("=" * 70)
    
#     # Load neighborhood data
#     logging.info(f"Loading neighborhood data from: {escrt_csv}")
#     escrt_df = pd.read_csv(escrt_csv)
#     escrt_df['genome_id_base'] = (
#     escrt_df['genome_id']
#     .str.split("_", n=2)
#     .str[0:2]
#     .str.join("_")
# )

#     logging.info(f"Neighborhood data: {len(escrt_df)} rows, {len(escrt_df['genome_id'].unique())} unique genomes")
    
#     # Load taxonomy data
#     tax_df = load_gtdb_taxonomy(gtdb_tsv)
    
#     # Check for overlap between datasets
#     escrt_genomes = set(escrt_df["genome_id_base"].unique())
#     tax_genomes = set(tax_df["genome_id_base"].unique())
#     overlap = escrt_genomes.intersection(tax_genomes)
    
#     logging.info(f"Genome ID overlap check:")
#     logging.info(f"  Genomes in neighborhood data: {len(escrt_genomes)}")
#     logging.info(f"  Genomes in taxonomy data: {len(tax_genomes)}")
#     logging.info(f"  Overlapping genomes: {len(overlap)}")
    
#     if len(overlap) == 0:
#         logging.error("NO OVERLAP between genome IDs!")
#         logging.error(f"Sample neighborhood genome IDs: {list(escrt_genomes)[:5]}")
#         logging.error(f"Sample taxonomy genome IDs: {list(tax_genomes)[:5]}")
#         logging.error("Check if genome_id formats match between files!")
#         sys.exit(1)
    
#     if len(overlap) < len(escrt_genomes):
#         missing = len(escrt_genomes) - len(overlap)
#         logging.warning(f"{missing} genomes from neighborhood data not found in taxonomy!")
    
#     # Perform the merge
#     logging.info("Performing left join on genome_id...")
#     merged = escrt_df.merge(
#         tax_df,
#         on="genome_id_base",
#         how="left"
#     )
    
#     # Validate merge results
#     logging.info(f"Merge complete: {len(merged)} rows")
    
#     # Check how many rows got taxonomy data
#     tax_columns = ["domain", "phylum", "class", "order", "family", "genus", "species"]
#     for col in tax_columns:
#         non_null = merged[col].notna().sum()
#         pct = (non_null / len(merged)) * 100
#         logging.info(f"  {col}: {non_null} non-null ({pct:.1f}%)")
    
#     # Save to CSV
#     logging.info(f"Saving merged data to: {out_csv}")
#     merged.to_csv(out_csv, index=False)
    
#     # Show sample of merged data
#     logging.info("Sample of merged data with taxonomy:")
#     sample_cols = ["genome_id", "protein_id", "domain", "phylum", "class", "order"]
#     available_cols = [col for col in sample_cols if col in merged.columns]
#     logging.info(merged[available_cols].head(10).to_string())
    
#     logging.info("=" * 70)
#     logging.info("TAXONOMY MERGE COMPLETE")
#     logging.info("=" * 70)
    
#     return merged




In [49]:

# ------------------------------------------------------------
# Parse Prodigal GFF and assign gene order PER CONTIG
# ------------------------------------------------------------
def parse_prodigal_gff(gff_path, genome_id):
    """
    Parse a Prodigal GFF3 file and extract CDS gene order per contig.

    Gene order is defined by appearance order in the GFF,
    which corresponds to genomic order for Prodigal output.
    
    Args:
        gff_path: Path to the GFF file
        genome_id: Identifier for the genome
        
    Returns:
        DataFrame with columns: genome_id, contig, gene_index, 
                                protein_id, start, end, strand
    """

    rows = []
    gene_counter = {}  # separate gene index per contig

    logging.info(f"Parsing GFF: {gff_path.name}")

    with open(gff_path) as f:
        for line in f:
            # Skip comment lines
            if line.startswith("#"):
                continue

            parts = line.rstrip().split("\t")
            if len(parts) != 9:
                continue

            contig, source, feature, start, end, score, strand, phase, attrs = parts

            # Only process CDS features
            if feature != "CDS":
                continue

            # Parse attributes to extract protein ID
            attr_dict = {}
            for item in attrs.split(";"):
                if "=" in item:
                    k, v = item.split("=", 1)
                    attr_dict[k] = v

            protein_id = attr_dict.get("ID")
            if protein_id is None:
                continue

            # Increment gene index PER CONTIG (not genome-wide)
            gene_counter.setdefault(contig, 0)
            gene_counter[contig] += 1

            rows.append({
                "genome_id": genome_id,
                "contig": contig,
                "gene_index": gene_counter[contig],
                "protein_id": protein_id,
                "start": int(start),
                "end": int(end),
                "strand": strand
            })

    logging.info(f"  → Parsed {len(rows)} CDS features across {len(gene_counter)} contigs")
    return pd.DataFrame(rows)


    

# ------------------------------------------------------------
# Load only the GFFs required by the hits table
# ------------------------------------------------------------
def load_gffs_from_hits(hits_df, gff_dir):
    """
    Parse GFF files only for genomes present in the hits dataframe.
    GFF filenames are inferred as: <genome_file>_genomic/<genome_file>_genomic.gff
    
    Args:
        hits_df: DataFrame containing hits with 'genome_file' column
        gff_dir: Directory containing GFF files
        
    Returns:
        Combined DataFrame of all parsed GFF files
    """

    all_gff_rows = []
    
    unique_genomes = hits_df["Name"].unique()
    logging.info(f"Loading GFFs for {len(unique_genomes)} unique genomes")

    for genome_id in unique_genomes:
        gff_path = gff_dir / f"{genome_id}_genomic" / f"{genome_id}_genomic.gff"

        if not gff_path.exists():
            logging.warning(f"Missing GFF: {gff_path}")
            continue

        gff_df = parse_prodigal_gff(gff_path, genome_id)
        all_gff_rows.append(gff_df)

    if not all_gff_rows:
        logging.error("No GFF files were successfully loaded.")
        sys.exit(1)

    combined_gff = pd.concat(all_gff_rows, ignore_index=True)
    logging.info(f"Total CDS features loaded: {len(combined_gff)}")
    
    return combined_gff


def build_windows(anchor_df, window):
    """
    Build genomic windows (start, end) around anchor genes.
    """
    windows = []

    for _, row in anchor_df.iterrows():
        windows.append({
            "genome_id": row["genome_id"],
            "contig": row["contig"],
            "winstart": row["gene_index"] - window,
            "winend": row["gene_index"] + window,
            "wincenter": row["gene_index"],
            "protein_id": row["protein_id"],
            "architecture": row["architecture"],
        })

    return pd.DataFrame(windows)




def merge_windows(windows_df):
    """
    Merge overlapping windows per genome/contig.
    """
    merged = []

    for (genome, contig), grp in windows_df.groupby(["genome_id", "contig"]):
        grp = grp.sort_values("winstart")

        cur_start = None
        cur_end = None
        cur_centers = []

        for _, row in grp.iterrows():
            if cur_start is None:
                cur_start = row["winstart"]
                cur_end = row["winend"]
                cur_centers = [row]
                continue

            if row["winstart"] <= cur_end + 1:
                # overlap → extend
                cur_end = max(cur_end, row["winend"])
                cur_centers.append(row)
            else:
                merged.append({
                    "genome_id": genome,
                    "contig": contig,
                    "winstart": cur_start,
                    "winend": cur_end,
                    "anchors": cur_centers
                })
                cur_start = row["winstart"]
                cur_end = row["winend"]
                cur_centers = [row]

        merged.append({
            "genome_id": genome,
            "contig": contig,
            "winstart": cur_start,
            "winend": cur_end,
            "anchors": cur_centers
        })

    return merged



def extract_merged_neighborhoods(merged_windows, gff_df):
    blocks = []

    for win in merged_windows:
        block = gff_df[
            (gff_df["genome_id"] == win["genome_id"]) &
            (gff_df["contig"] == win["contig"]) &
            (gff_df["gene_index"].between(win["winstart"], win["winend"]))
        ].copy()

        if block.empty:
            continue

        # pick a representative anchor (central-most)
        centers = [a["wincenter"] for a in win["anchors"]]
        rep_center = min(centers, key=lambda x: abs(x - (win["winstart"] + win["winend"]) / 2))

        block["relative_pos"] = block["gene_index"] - rep_center
        block["center_protein"] = win["anchors"][0]["protein_id"]
        block["center_gene"] = win["anchors"][0]["architecture"]

        blocks.append(block)

    return pd.concat(blocks, ignore_index=True)



def extract_neighborhoods(anchor_df, gff_df, window=5):

    keywords = ("vps", "escrt", "katanin", "eap", "flad", "zn-ph")

    logging.info(f"Starting neighborhood extraction (±{window} genes)")
    logging.info(f"Initial anchors: {len(anchor_df)}")

    filtered_anchors = anchor_df[
        anchor_df["architecture"]
        .astype(str)
        .str.lower()
        .str.contains("|".join(keywords))
    ]

    logging.info(f"Anchors after ESCRT filter: {len(filtered_anchors)}")

    if filtered_anchors.empty:
        logging.error("No ESCRT-related anchors found")
        sys.exit(1)

    windows_df = build_windows(filtered_anchors, window)
    merged_windows = merge_windows(windows_df)

    logging.info(
        f"Merged {len(windows_df)} anchor windows into "
        f"{len(merged_windows)} non-overlapping regions"
    )

    combined_neighborhoods = extract_merged_neighborhoods(
        merged_windows, gff_df
    )

    logging.info(
        f"Extracted {len(combined_neighborhoods)} genes "
        f"from {len(merged_windows)} merged neighborhoods"
    )

    return combined_neighborhoods


# ------------------------------------------------------------
# Load GTDB taxonomy
# ------------------------------------------------------------
def load_gtdb_taxonomy(gtdb_tsv):
    """
    Load GTDB taxonomy file and split into rank-specific columns.
    
    GTDB format: genome_id\td__Domain;p__Phylum;c__Class;o__Order;f__Family;g__Genus;s__Species
    
    Args:
        gtdb_tsv: Path to GTDB taxonomy TSV file
        
    Returns:
        DataFrame with columns: genome_id, domain, phylum, class, order, family, genus, species
    """
    logging.info(f"Loading GTDB taxonomy from: {gtdb_tsv}")
    
    # Read the taxonomy file
    tax_df = pd.read_csv(
        gtdb_tsv,
        sep="\t",
        header=None,
        names=["genome_id", "taxonomy"],
        dtype=str
    )
    
    tax_df["genome_id_base"] = tax_df["genome_id"].str.split("_", n=1).str[1]
    logging.info(f"Loaded taxonomy for {len(tax_df)} genomes")
    logging.info(f"Sample taxonomy entry: {tax_df['taxonomy'].iloc[0]}")
    
    # Split taxonomy string by semicolons
    tax_split = tax_df["taxonomy"].str.split(";", expand=True)
    logging.info(f"Taxonomy split into {tax_split.shape[1]} columns")
    
    # Map column indices to taxonomic ranks
    rank_map = {
        0: "domain",
        1: "phylum",
        2: "class",
        3: "order",
        4: "family",
        5: "genus",
        6: "species"
    }
    
    # Extract each rank and remove the prefix (e.g., "d__", "p__")
    for idx, rank in rank_map.items():
        if idx < tax_split.shape[1]:
            # Remove the rank prefix using regex (e.g., "p__" from "p__Crenarchaeota")
            tax_df[rank] = tax_split[idx].str.replace(r"^[a-z]__", "", regex=True)
            logging.info(f"Extracted {rank}: {tax_df[rank].notna().sum()} non-null values")
        else:
            tax_df[rank] = pd.NA
            logging.warning(f"Column {idx} for rank '{rank}' not found in taxonomy data")
    
    # Drop the original concatenated taxonomy string
    tax_df.drop(columns=["taxonomy"], inplace=True)
    
    # Log sample of parsed taxonomy
    logging.info(f"Sample parsed taxonomy:")
    logging.info(tax_df[["genome_id", "domain", "phylum", "class"]].head().to_string())
    
    return tax_df


# ------------------------------------------------------------
# Merge taxonomy into neighborhood data
# ------------------------------------------------------------
def merge_taxonomy(escrt_csv, gtdb_tsv, out_csv):
    """
    Merge GTDB taxonomy into ESCRT neighborhood dataframe.
    
    Args:
        escrt_csv: Path to ESCRT neighborhoods CSV
        gtdb_tsv: Path to GTDB taxonomy TSV
        out_csv: Output path for merged CSV
        
    Returns:
        Merged DataFrame with taxonomy columns added
    """
    logging.info("=" * 70)
    logging.info("STARTING TAXONOMY MERGE")
    logging.info("=" * 70)
    
    # Load neighborhood data
    logging.info(f"Loading neighborhood data from: {escrt_csv}")
    escrt_df = pd.read_csv(escrt_csv)
    escrt_df['genome_id_base'] = (
    escrt_df['genome_id']
    .str.split("_", n=2)
    .str[0:2]
    .str.join("_")
)

    logging.info(f"Neighborhood data: {len(escrt_df)} rows, {len(escrt_df['genome_id'].unique())} unique genomes")
    
    # Load taxonomy data
    tax_df = load_gtdb_taxonomy(gtdb_tsv)
    
    # Check for overlap between datasets
    escrt_genomes = set(escrt_df["genome_id_base"].unique())
    tax_genomes = set(tax_df["genome_id_base"].unique())
    overlap = escrt_genomes.intersection(tax_genomes)
    
    logging.info(f"Genome ID overlap check:")
    logging.info(f"  Genomes in neighborhood data: {len(escrt_genomes)}")
    logging.info(f"  Genomes in taxonomy data: {len(tax_genomes)}")
    logging.info(f"  Overlapping genomes: {len(overlap)}")
    
    if len(overlap) == 0:
        logging.error("NO OVERLAP between genome IDs!")
        logging.error(f"Sample neighborhood genome IDs: {list(escrt_genomes)[:5]}")
        logging.error(f"Sample taxonomy genome IDs: {list(tax_genomes)[:5]}")
        logging.error("Check if genome_id formats match between files!")
        sys.exit(1)
    
    if len(overlap) < len(escrt_genomes):
        missing = len(escrt_genomes) - len(overlap)
        logging.warning(f"{missing} genomes from neighborhood data not found in taxonomy!")
    
    # Perform the merge
    logging.info("Performing left join on genome_id...")
    merged = escrt_df.merge(
        tax_df,
        on="genome_id_base",
        how="left"
    )
    
    # Validate merge results
    logging.info(f"Merge complete: {len(merged)} rows")
    
    # Check how many rows got taxonomy data
    tax_columns = ["domain", "phylum", "class", "order", "family", "genus", "species"]
    for col in tax_columns:
        non_null = merged[col].notna().sum()
        pct = (non_null / len(merged)) * 100
        logging.info(f"  {col}: {non_null} non-null ({pct:.1f}%)")
    
    # Save to CSV
    logging.info(f"Saving merged data to: {out_csv}")
    merged.to_csv(out_csv, index=False)
    
    # Show sample of merged data
    logging.info("Sample of merged data with taxonomy:")
    sample_cols = ["genome_id", "protein_id", "domain", "phylum", "class", "order"]
    available_cols = [col for col in sample_cols if col in merged.columns]
    logging.info(merged[available_cols].head(10).to_string())
    
    logging.info("=" * 70)
    logging.info("TAXONOMY MERGE COMPLETE")
    logging.info("=" * 70)
    
    return merged




In [50]:
from pathlib import Path

# ------------------------------------------------------------
# Main pipeline
# ------------------------------------------------------------
def run_synteny_pipeline(
    hits_df,
    gff_dir,
    window=5,
    min_coverage=0.65,
    max_i_evalue=1e-5,
):

    logging.info("=" * 70)
    logging.info("STARTING SYNTENY PIPELINE")
    logging.info("=" * 70)
    logging.info(f"Hits DataFrame: {hits_df}")
    logging.info(f"GFF directory: {gff_dir}")
    logging.info(f"Window size: ±{window} genes")


    gff_dir = Path(gff_dir)


    # --------------------------------------------------------
    # Load GFFs
    # --------------------------------------------------------
    logging.info("\n[STEP 1] Loading GFF files...")
    gff_df = load_gffs_from_hits(hits_df, gff_dir)


    gff_df.to_csv(os.path.join(MAIN_OUTDIR, f"[STEP:{STEP}.5]gff_dataframe_{rightnow}.csv"), index=False)

    # --------------------------------------------------------
    # Map anchor hits to gene order
    # --------------------------------------------------------
    logging.info("\n[STEP 2] Mapping hits to gene order...")
    anchor_df = hits_df.merge(
        gff_df,
        left_on="target",
        right_on="protein_id",
        how="inner"
    )


    if anchor_df.empty:
        logging.error("No hits could be mapped to GFFs (protein ID mismatch?)")
        sys.exit(1)

    protein_to_arch = (
    anchor_df
    .drop_duplicates("protein_id")
    .set_index("protein_id")["architecture"]
    .to_dict()
    )



    # --------------------------------------------------------
    # Extract neighborhoods
    # --------------------------------------------------------
    logging.info("\n[STEP 3] Extracting gene neighborhoods...")
    neigh_df = extract_neighborhoods(anchor_df, gff_df, window=window)

    # --------------------------------------------------------
    # Annotate EACH neighbor with its own query identity
    # --------------------------------------------------------
    logging.info("\n[STEP 4] Annotating neighbors with query IDs...")
    neigh_df["neighbor_architecture"] = neigh_df["protein_id"].map(protein_to_arch)
    neigh_df["is_target_family"] = neigh_df["neighbor_architecture"].notna()

    # Debug: Check neighbor_query values
    logging.info(f"Total genes in neighborhoods: {len(neigh_df)}")
    logging.info(f"Genes with neighbor_query annotation: {neigh_df['neighbor_architecture'].notna().sum()}")
    logging.info(f"Sample neighbor_query values: {neigh_df['neighbor_architecture'].dropna().unique()[:10].tolist()}")


    # --------------------------------------------------------
    # Merge center gene annotations - WITH DEBUGGING
    # --------------------------------------------------------
    logging.info("\n[STEP 5] Merging center gene AsCOG annotations...")

    # Strip whitespace from center_gene
    neigh_df["center_gene"] = neigh_df["center_gene"].str.strip()


    logging.info(
        f"\nAnnotated neighborhoods: {len(neigh_df)} total genes"
    )

    logging.info("=" * 70)
    logging.info("SYNTENY PIPELINE COMPLETED SUCCESSFULLY")
    logging.info("=" * 70)

    return neigh_df, anchor_df




In [51]:
# Run the main synteny pipeline
neigh_df, anchor_df = run_synteny_pipeline(
    hits_df = df_with_arch,
    gff_dir="/home/anirudh/genomes/selected_genomes/prokka_results",
    window=5,
    min_coverage=0.65,
    max_i_evalue=1e-5,
)

# Quick sanity checks
logging.info("\n" + "=" * 70)
logging.info("PIPELINE OUTPUT SUMMARY")
logging.info("=" * 70)

logging.info("\nAnchor gene Architecture distribution:")
logging.info(anchor_df["architecture"].value_counts().to_string())

logging.info("\nFirst few neighborhood entries:")
logging.info(neigh_df.head().to_string())

logging.info("\nGenome distribution:")
logging.info(neigh_df['genome_id'].value_counts().to_string())


# Save intermediate results
logging.info("\nSaving neighborhood data...")
STEP += 1

escrt_nieghborhoods_file = os.path.join(MAIN_OUTDIR, f"[STEP:{STEP}]escrt_neighborhoods_{rightnow}.csv")
neigh_df.to_csv(escrt_nieghborhoods_file, index=False)
logging.info(f"Saved: [STEP:{STEP}]escrt_neighborhoods_{rightnow}.csv")

logging.info("Saving anchor hits...")
STEP += 1
anchor_df.to_csv(os.path.join(MAIN_OUTDIR, f"[STEP:{STEP}]escrt_anchor_hits_{rightnow}.csv"), index=False)
logging.info(f"Saved: [STEP:{STEP}]escrt_anchor_hits_{rightnow}.csv")

# Merge with taxonomy
logging.info("\n" + "=" * 70)
logging.info("ADDING TAXONOMY INFORMATION")
logging.info("=" * 70)

STEP += 1
out_csv = os.path.join(MAIN_OUTDIR, f"[STEP:{STEP}]escrt_neighborhoods_with_taxonomy_{rightnow}.csv")
df = merge_taxonomy(
    escrt_nieghborhoods_file, 
    "/home/anirudh/synteny/ar53_taxonomy_r226.tsv", 
    out_csv
)

# Show taxonomic diversity in results   
logging.info("\nTaxonomic diversity in results:")
logging.info(f"Unique phyla: {df['phylum'].nunique()}")
logging.info(f"Unique classes: {df['class'].nunique()}") 
logging.info(f"Unique orders: {df['order'].nunique()}")

logging.info("\nSample genomes with taxonomy:")
logging.info(df[["genome_id_base", "phylum", "class", "order"]].drop_duplicates().head(10).to_string())

logging.info("\n" + "=" * 70)
logging.info("ALL PROCESSING COMPLETE")
logging.info("=" * 70)

2026-01-21 11:08:19,731 [INFO] ======================================================================
2026-01-21 11:08:19,732 [INFO] STARTING SYNTENY PIPELINE
2026-01-21 11:08:19,733 [INFO] ======================================================================
2026-01-21 11:08:19,735 [INFO] Hits DataFrame:               target                          Name  Completeness  \
0     AAEOKEPF_00166  GCA_021498095.1_ASM2149809v1         93.44   
3     AAEOKEPF_00273  GCA_021498095.1_ASM2149809v1         93.44   
6     AAEOKEPF_00322  GCA_021498095.1_ASM2149809v1         93.44   
8     AAEOKEPF_00337  GCA_021498095.1_ASM2149809v1         93.44   
10    AAEOKEPF_00338  GCA_021498095.1_ASM2149809v1         93.44   
...              ...                           ...           ...   
6669  PLALBJDB_03422  GCA_019058015.1_ASM1905801v1         92.80   
6674  PLALBJDB_03434  GCA_019058015.1_ASM1905801v1         92.80   
6681  PLALBJDB_03435  GCA_019058015.1_ASM1905801v1         92.80   
6688  PLALBJ

In [52]:
# ------------------------------------------------------------------
# DIAGNOSTIC: Check which architectures would be filtered by keywords
# ------------------------------------------------------------------
keywords = ("vps", "escrt", "katanin", "eap", "flad")
keyword_pattern = "|".join(keywords)

# Create a boolean mask for architectures matching keywords
matches_keywords = (
    df_with_arch["architecture"]
    .astype(str)
    .str.lower()
    .str.contains(keyword_pattern)
)

# Count what would be kept vs discarded
kept_count = matches_keywords.sum()
discarded_count = (~matches_keywords).sum()

logging.info("\n" + "=" * 70)
logging.info("ARCHITECTURE FILTERING DIAGNOSTIC")
logging.info("=" * 70)
logging.info(f"Total proteins with valid HMM hits (i_evalue ≤ 1e-5, coverage ≥ 0.65): {len(df_with_arch)}")
logging.info(f"Proteins with architectures matching keywords: {kept_count}")
logging.info(f"Proteins with architectures NOT matching keywords: {discarded_count}")

if discarded_count > 0:
    logging.warning(f"\n⚠️  WARNING: {discarded_count} proteins will be EXCLUDED from neighborhoods:")
    excluded = df_with_arch[~matches_keywords][["target", "architecture"]].drop_duplicates()
    logging.warning(excluded.head(20).to_string())
    
    # Count distribution of excluded architectures
    logging.warning("\nDistribution of excluded architectures:")
    arch_counts = df_with_arch[~matches_keywords]["architecture"].value_counts()
    logging.warning(arch_counts.head(20).to_string())
    
    # Save for review
    excluded_file = os.path.join(MAIN_OUTDIR, f"[DIAGNOSTIC]excluded_proteins_by_keyword_filter_{rightnow}.csv")
    df_with_arch[~matches_keywords][["target", "architecture", "architecture_method"]].drop_duplicates().sort_values(by = ['architecture'], ascending=False).to_csv(excluded_file, index=False)
    logging.warning(f"\nSaved excluded proteins to: {excluded_file}")
else:
    logging.info("✓ All proteins with valid hits match keyword filter - no data loss")

logging.info("=" * 70)

2026-01-21 11:08:43,109 [INFO] 
2026-01-21 11:08:43,110 [INFO] ARCHITECTURE FILTERING DIAGNOSTIC
2026-01-21 11:08:43,111 [INFO] ======================================================================
2026-01-21 11:08:43,111 [INFO] Total proteins with valid HMM hits (i_evalue ≤ 1e-5, coverage ≥ 0.65): 3325
2026-01-21 11:08:43,112 [INFO] Proteins with architectures matching keywords: 1470
2026-01-21 11:08:43,112 [INFO] Proteins with architectures NOT matching keywords: 1855
2026-01-21 11:08:43,113 [WARNING] 
⚠️  WARNING: 1855 proteins will be EXCLUDED from neighborhoods:
2026-01-21 11:08:43,117 [WARNING]             target architecture
0   AAEOKEPF_00166           E2
3   AAEOKEPF_00273       E3-UFM
6   AAEOKEPF_00322           E1
8   AAEOKEPF_00337           E1
10  AAEOKEPF_00338          MPN
12  AAEOKEPF_00690          MPN
13  AAEOKEPF_00878          MPN
15  AAEOKEPF_00882      Ub-like
19  AAEOKEPF_01109          MPN
20  AAEOKEPF_01385          MPN
29  AAEOKEPF_01448          MPN
38  AAE

In [53]:
# Architecture Analysis

#!/usr/bin/env python3
"""
Architecture analysis and reduction script (orientation-invariant)
=================================================================

Input:
  - Annotated neighborhood CSV with taxonomy and architecture labels

Output:
  - Contig-level trimmed & canonical architectures
  - Lineage-level architecture counts (inverse + subtuple collapsed)
  - Representative contigs per lineage
  - Gene-level dataframe for plotting

Key features:
  - Flank trimming (k=2)
  - Orientation-invariant (inverse) collapsing
  - Counts incorporate reversed architectures
  - Subtuple matching for architecture equivalence
"""

import pandas as pd
import logging
import os
from pathlib import Path
from datetime import datetime

# ------------------------------------------------------------------
# CONFIG
# ------------------------------------------------------------------
INPUT_CSV = out_csv # Output from previous step with taxonomy [should be [step:8]escrt_neighborhoods_with_taxonomy_{rightnow}.csv]
OUTDIR = Path(os.path.join(MAIN_OUTDIR, f"architecture_results_{rightnow}"))
OUTDIR.mkdir(exist_ok=True)

# ------------------------------------------------------------------
# Helper functions
# ------------------------------------------------------------------
def trim_architecture(tokens, k=2):
    """
    Trim architecture by keeping at most k 'other' tokens
    at each flank while retaining internal structure.
    """
    tokens = list(tokens)

    left = 0
    while left < len(tokens) and tokens[left] == "other":
        left += 1
    left = max(0, left - k)

    right = len(tokens)
    while right > 0 and tokens[right - 1] == "other":
        right -= 1
    right = min(len(tokens), right + k)

    return tuple(tokens[left:right])


def canonicalize_architecture(tokens):
    """
    Orientation-invariant canonical form
    """
    tokens = tuple(tokens)
    return min(tokens, tokens[::-1])


def is_subtuple(short, long):
    """
    Check if `short` is a contiguous subtuple of `long`
    """
    n, m = len(short), len(long)
    if n > m:
        return False
    for i in range(m - n + 1):
        if long[i:i+n] == short:
            return True
    return False


def architecture_equivalent(a, b):
    """
    Equivalence under:
      - inversion
      - truncation (subtuple)
      - inversion + truncation
    """
    a = tuple(a)
    b = tuple(b)

    return (
        is_subtuple(a, b) or
        is_subtuple(a, b[::-1]) or
        is_subtuple(a[::-1], b) or
        is_subtuple(a[::-1], b[::-1])
    )

# ------------------------------------------------------------------
# Step 1: Load data
# ------------------------------------------------------------------
logging.info("Loading annotated neighborhood table")
df = pd.read_csv(INPUT_CSV)

df["neighbor_architecture"] = df["neighbor_architecture"].str.strip()

required_cols = {
    "genome_id_x", "contig", "center_protein",
    "relative_pos", "neighbor_architecture",
    LINEAGE_LEVEL
}
missing = required_cols - set(df.columns)
if missing:
    raise ValueError(f"Missing required columns: {missing}")

logging.info(f"Loaded {len(df)} gene-level rows")

# ------------------------------------------------------------------
# Step 2: Encode architecture tokens
# ------------------------------------------------------------------
logging.info("Encoding architecture tokens")

df["arch_token"] = df["neighbor_architecture"].fillna("other")

# ------------------------------------------------------------------
# Step 3: Build contig-level architectures
# ------------------------------------------------------------------
logging.info("Constructing contig-level architectures")

arch_df = (
    df
    .sort_values("relative_pos")
    .groupby(
        ["genome_id_x", "contig", "center_protein", LINEAGE_LEVEL],
        as_index=False
    )
    .agg(
        arch_token=("arch_token", lambda x: tuple(x)),
        n_genes=("arch_token", "count")
    )
)

logging.info(f"Identified {len(arch_df)} contig-level architectures")

STEP += 1
arch_df.to_csv(
    OUTDIR / f"[STEP:{STEP}]contig_architectures_raw.csv",
    index=False
)

# ------------------------------------------------------------------
# Step 4: Trim flanks + canonicalize
# ------------------------------------------------------------------
logging.info(f"Trimming architectures (k={FLANK_K}) and collapsing inverses")

arch_df["arch_token_trimmed"] = arch_df["arch_token"].apply(
    lambda x: trim_architecture(x, k=FLANK_K)
)

arch_df["arch_token_canonical"] = arch_df["arch_token_trimmed"].apply(
    canonicalize_architecture
)

arch_df["trimmed_len"] = arch_df["arch_token_trimmed"].apply(len)

STEP += 1
arch_df.to_csv(
    OUTDIR / f"[STEP:{STEP}]contig_architectures_trimmed_canonical.csv",
    index=False
)

# ------------------------------------------------------------------
# Step 5A: Collapse to maximal architectures per lineage
# ------------------------------------------------------------------
logging.info("Collapsing architectures using inversion + subtuple equivalence")

collapsed_rows = []
representatives = {}  # lineage -> list of maximal architectures

for lineage, subdf in arch_df.groupby(LINEAGE_LEVEL):
    subdf = subdf.copy()
    subdf["arch_len"] = subdf["arch_token_trimmed"].apply(len)
    subdf = subdf.sort_values("arch_len", ascending=False)

    kept_archs = []

    for _, row in subdf.iterrows():
        arch = row["arch_token_trimmed"]

        if any(architecture_equivalent(arch, kept) for kept in kept_archs):
            continue

        kept_archs.append(arch)
        collapsed_rows.append(row)

    representatives[lineage] = kept_archs

collapsed_arch_df = pd.DataFrame(collapsed_rows).drop(columns="arch_len")

STEP += 1
collapsed_arch_df.to_csv(
    OUTDIR / f"[STEP:{STEP}]collapsed_arch_df.csv",
    index=False
)

# ------------------------------------------------------------------
# Step 5B: Assign EVERY architecture to a representative
# ------------------------------------------------------------------
logging.info(
    "Assigning architectures to maximal representatives (subtuple + inverse-aware)"
)

def assign_representative(row):
    lineage = row[LINEAGE_LEVEL]
    arch = row["arch_token_trimmed"]

    for rep in representatives[lineage]:
        if architecture_equivalent(arch, rep):
            return canonicalize_architecture(rep)

    return None

arch_df["rep_arch_canonical"] = arch_df.apply(assign_representative, axis=1)

if arch_df["rep_arch_canonical"].isna().any():
    raise RuntimeError("Some architectures could not be assigned to a representative")

STEP += 1
arch_df.to_csv(
    OUTDIR / f"[STEP:{STEP}]contig_architectures_with_representatives.csv",
    index=False
)

# ------------------------------------------------------------------
# Step 5C: Count INCLUDING subtuples
# ------------------------------------------------------------------
logging.info(
    f"Counting architectures per {LINEAGE_LEVEL} (inverse + subtuple collapsed)"
)

arch_counts = (
    arch_df
    .groupby([LINEAGE_LEVEL, "rep_arch_canonical"], as_index=False)
    .size()
    .rename(columns={"size": "count"})
    .sort_values([LINEAGE_LEVEL, "count"], ascending=[True, False])
)

STEP += 1
arch_counts.to_csv(
    OUTDIR / f"[STEP:{STEP}]architecture_frequencies_inverse_subtuple_collapsed.csv",
    index=False
)



# ------------------------------------------------------------------------------
# Step 6: Select representative architecture per lineage
# ------------------------------------------------------------------------------
logging.info("Selecting representative architecture per lineage")


# doesnt consider singlets for representative selection
arch_counts["information"] = arch_counts["rep_arch_canonical"].apply(
    lambda arch: sum(gene != "other" for gene in arch)
)
logging.info(arch_counts[["rep_arch_canonical", "information", "count"]].head(10))


rep_arch = (
    arch_counts
    .query("information >= 2")
    .sort_values(["information", "count"], ascending=[False, False])
    .groupby(LINEAGE_LEVEL, as_index=False)
    .head(1)
)




STEP += 1
rep_arch.to_csv(
    OUTDIR / f"[STEP:{STEP}]representative_architectures.csv",
    index=False
)

logging.info(f"Selected {len(rep_arch)} representative architectures")


# ------------------------------------------------------------------
# Step 7: Recover genomes encoding the longest representative architectures
# ------------------------------------------------------------------
logging.info(
    "Recovering genomes encoding longest architectures with maximal subtuple support"
)

# Only maximal (longest) architectures per lineage
longest_arch_contigs = collapsed_arch_df[[
    LINEAGE_LEVEL,
    "genome_id_x",
    "contig",
    "center_protein",
    "arch_token_canonical",
    "arch_token_trimmed"
]].copy()

# Merge architecture counts → longest architecture contigs
rep_keys = rep_arch.merge(
    longest_arch_contigs,
    left_on=[LINEAGE_LEVEL, "rep_arch_canonical"],
    right_on=[LINEAGE_LEVEL, "arch_token_canonical"],
    how="left"
)

# Defensive check: every representative should map to ≥1 genome
if rep_keys["genome_id_x"].isna().any():
    raise RuntimeError(
        "Some representative architectures could not be mapped to a genome"
    )

STEP += 1
rep_keys.to_csv(
    OUTDIR / f"[STEP:{STEP}]representative_architectures_with_genomes.csv",
    index=False
)
# # ------------------------------------------------------------------
# # Step 8: Recover gene-level rows for plotting (representative architectures only)
# # ------------------------------------------------------------------
# logging.info("Extracting gene-level rows for plotting (representative architectures)")

# logging.info("Columns in input df: %s", str(", ".join(df.columns.tolist())))

# plot_df = df.merge(
#     rep_keys[
#         [
#             LINEAGE_LEVEL,
#             "genome_id_x",
#             "contig",
#             "center_protein",
#             "rep_arch_canonical"
#         ]
#     ],
#     on=[LINEAGE_LEVEL, "genome_id_x"],
#     how="inner"
# )
# logging.info(
#     "Columns in plot df: %s",
#     ", ".join(plot_df.columns.tolist())
# )

# plot_df = plot_df.sort_values(
#     [LINEAGE_LEVEL, "genome_id_x", "contig_x", "relative_pos"]
# )

# STEP += 1
# plot_df.to_csv(
#     OUTDIR / f"[STEP:{STEP}]plot_ready_architectures.csv",
#     index=False
# )


# ------------------------------------------------------------------
# Step 8: Recover gene-level rows for plotting
#   - include ALL contigs from representative genomes
#   - ensure at least one contig encodes the representative architecture
# ------------------------------------------------------------------
logging.info(
    "Extracting gene-level rows for plotting (all contigs from representative genomes)"
)

# 1. Identify representative genomes per lineage
print("rep_keys columns: ", rep_keys.columns)


rep_genomes = (
    rep_keys[
        [
            LINEAGE_LEVEL,
            "genome_id_x",
            "rep_arch_canonical",
            "contig"
        ]
    ]
    .drop_duplicates()
)


STEP += 1
rep_genomes.to_csv(
    OUTDIR / f"[STEP:{STEP}]representative_genomes.csv",
    index=False
)

logging.info(
    "Representative genomes identified: %d",
    len(rep_genomes)
)

# 2. Pull ALL gene-level rows from those genomes
plot_df = df.merge(
    rep_genomes,
    on=[LINEAGE_LEVEL, "genome_id_x", "contig"],
    how="inner"
)

logging.info(
    "Plot df shape after genome-level filtering: %s",
    plot_df.shape
)

# 3. Sort for clean plotting
plot_df = plot_df.sort_values(
    [LINEAGE_LEVEL, "genome_id_x", "contig", "gene_index"]
)

STEP += 1
plot_df.to_csv(
    OUTDIR / f"[STEP:{STEP}]plot_ready_architectures.csv",
    index=False
)


# Summary
# ------------------------------------------------------------------
logging.info("Architecture analysis complete")
logging.info(f"Results written to: {OUTDIR.resolve()}")

logging.info("Files generated:")
for f in sorted(OUTDIR.iterdir()):
    logging.info(f"  - {f.name}")


2026-01-21 11:08:43,140 [INFO] Loading annotated neighborhood table
2026-01-21 11:08:43,155 [INFO] Loaded 9312 gene-level rows
2026-01-21 11:08:43,155 [INFO] Encoding architecture tokens
2026-01-21 11:08:43,156 [INFO] Constructing contig-level architectures
2026-01-21 11:08:43,166 [INFO] Identified 574 contig-level architectures
2026-01-21 11:08:43,169 [INFO] Trimming architectures (k=0) and collapsing inverses
2026-01-21 11:08:43,173 [INFO] Collapsing architectures using inversion + subtuple equivalence
2026-01-21 11:08:43,193 [INFO] Assigning architectures to maximal representatives (subtuple + inverse-aware)
2026-01-21 11:08:43,200 [INFO] Counting architectures per family (inverse + subtuple collapsed)
2026-01-21 11:08:43,203 [INFO] Selecting representative architecture per lineage
2026-01-21 11:08:43,204 [INFO]                                   rep_arch_canonical  information  count
0  (EAP30, Vps28, Vps28, E2-VPS23, ESCRTIII, ESCR...            9     16
1        (CC-VPS23, ESCRTII

In [54]:
# #!/usr/bin/env python3

# # without count for subtuples
# """
# Architecture analysis and reduction script (orientation-invariant)
# =================================================================

# Input:
#   - Annotated neighborhood CSV with taxonomy and architecture labels

# Output:
#   - Contig-level trimmed & canonical architectures
#   - Lineage-level architecture counts (inverse-collapsed)
#   - Representative contigs per lineage
#   - Gene-level dataframe for plotting

# Key features:
#   - Flank trimming (k=2)
#   - Orientation-invariant (inverse) collapsing
#   - Counts incorporate reversed architectures
#   - Subtuple matching for architecture equivalence
# """

# import pandas as pd
# import logging
# from pathlib import Path
# from datetime import datetime

# INPUT_CSV = out_csv # Output from previous step with taxonomy [should be [step:8]escrt_neighborhoods_with_taxonomy_{rightnow}.csv]
# OUTDIR = Path(os.path.join(MAIN_OUTDIR, f"architecture_results_{rightnow}"))


# OUTDIR.mkdir(exist_ok=True)

# # ------------------------------------------------------------------------------
# # Helper functions
# # ------------------------------------------------------------------------------
# def trim_architecture(tokens, k=2):
#     """
#     Trim architecture by keeping at most k 'other' tokens
#     at each flank while retaining internal structure.
#     """
#     tokens = list(tokens)

#     # Trim left
#     left = 0
#     while left < len(tokens) and tokens[left] == "other":
#         left += 1
#     left = max(0, left - k)

#     # Trim right
#     right = len(tokens)
#     while right > 0 and tokens[right - 1] == "other":
#         right -= 1
#     right = min(len(tokens), right + k)

#     return tuple(tokens[left:right])


# def canonicalize_architecture(tokens):
#     """
#     Make architecture orientation-invariant by collapsing
#     forward and reverse into a single canonical form.
#     """
#     tokens = tuple(tokens)
#     rev = tokens[::-1]
#     return min(tokens, rev)


# def is_subtuple(short, long):
#     """
#     Check if tuple `short` is a contiguous subtuple of `long`
#     """
#     n, m = len(short), len(long)
#     if n > m:
#         return False
#     for i in range(m - n + 1):
#         if long[i:i+n] == short:
#             return True
#     return False


# def architecture_equivalent(a, b):
#     """
#     True if architectures a and b are equivalent under:
#       - inversion
#       - truncation (subtuple)
#       - inversion + truncation
#     """
#     a = tuple(a)
#     b = tuple(b)

#     a_rev = a[::-1]
#     b_rev = b[::-1]

#     return (
#         is_subtuple(a, b) or
#         is_subtuple(a, b_rev) or
#         is_subtuple(a_rev, b) or
#         is_subtuple(a_rev, b_rev)
#     )



# # ------------------------------------------------------------------------------
# # Step 1: Load data
# # ------------------------------------------------------------------------------
# logging.info("Loading annotated neighborhood table")
# df = pd.read_csv(INPUT_CSV)


# df['neighbor_architecture'] = df['neighbor_architecture'].str.strip()
# required_cols = {
#     "genome_id_x", "contig", "center_protein",
#     "relative_pos", "neighbor_architecture",
#     LINEAGE_LEVEL
# }

# missing = required_cols - set(df.columns)
# if missing:
#     raise ValueError(f"Missing required columns: {missing}")

# logging.info(f"Loaded {len(df)} gene-level rows")

# # ------------------------------------------------------------------------------
# # Step 2: Encode architecture tokens
# # ------------------------------------------------------------------------------
# logging.info("Encoding architecture tokens")

# df["arch_token"] = (
#     df["neighbor_architecture"]
#     .fillna("other")
# )

# # ------------------------------------------------------------------------------
# # Step 3: Build contig-level architectures
# # ------------------------------------------------------------------------------
# logging.info("Constructing contig-level architectures")

# arch_df = (
#     df
#     .sort_values("relative_pos")
#     .groupby(
#         ["genome_id_x", "contig", "center_protein", LINEAGE_LEVEL],
#         as_index=False
#     )
#     .agg(
#         arch_token=("arch_token", lambda x: tuple(x)),
#         n_genes=("arch_token", "count")
#     )
# )

# logging.info(f"Identified {len(arch_df)} contig-level architectures")

# STEP += 1
# arch_df.to_csv(OUTDIR / f"[STEP:{STEP}]contig_architectures_raw.csv", index=False)

# # ------------------------------------------------------------------------------
# # Step 4: Trim flanks + inverse collapse (CORE STEP)
# # ------------------------------------------------------------------------------
# logging.info(
#     f"Trimming architectures (k={FLANK_K}) and collapsing inverses"
# )

# arch_df["arch_token_trimmed"] = arch_df["arch_token"].apply(
#     lambda x: trim_architecture(x, k=FLANK_K)
# )

# arch_df["arch_token_canonical"] = arch_df["arch_token_trimmed"].apply(
#     canonicalize_architecture
# )

# arch_df["trimmed_len"] = arch_df["arch_token_trimmed"].apply(len)


# STEP += 1
# arch_df.to_csv(
#     OUTDIR / f"[STEP:{STEP}]contig_architectures_trimmed_canonical.csv",
#     index=False
# )

# # ------------------------------------------------------------------------------
# # Step 5: Count architectures by lineage (INVERSE-AWARE)
# # ------------------------------------------------------------------------------


# logging.info("Collapsing architectures using inversion + subtuple equivalence")

# collapsed_rows = []

# for lineage, subdf in arch_df.groupby(LINEAGE_LEVEL):
#     subdf = subdf.copy()
#     subdf["arch_len"] = subdf["arch_token_trimmed"].apply(len)

#     # longest-first ensures maximal architecture survives
#     subdf = subdf.sort_values("arch_len", ascending=False)

#     kept_archs = []

#     for _, row in subdf.iterrows():
#         arch = row["arch_token_trimmed"]

#         if any(architecture_equivalent(arch, kept) for kept in kept_archs):
#             continue

#         kept_archs.append(arch)
#         collapsed_rows.append(row)

# collapsed_arch_df = pd.DataFrame(collapsed_rows).drop(columns="arch_len")

# STEP += 1

# collapsed_arch_df.to_csv(
#     OUTDIR / f"[STEP:{STEP}]collapsed_arch_df.csv",
#     index=False
# )


# logging.info(
#     f"Counting architectures per {LINEAGE_LEVEL} (inverse-aware)"
# )

# arch_counts = (
#     collapsed_arch_df
#     .groupby([LINEAGE_LEVEL, "arch_token_canonical"], as_index=False)
#     .size()
#     .rename(columns={"size": "count"})
#     .sort_values([LINEAGE_LEVEL, "count"], ascending=[True, False])
# )

# STEP += 1

# arch_counts.to_csv(
#     OUTDIR / f"[STEP:{STEP}]architecture_frequencies_inverse_collapsed.csv",
#     index=False
# )

# # # ------------------------------------------------------------------------------
# # # Step 6: Select representative architecture per lineage
# # # ------------------------------------------------------------------------------
# # logging.info("Selecting representative architecture per lineage")

# # rep_arch = (
# #     arch_counts
# #     .groupby(LINEAGE_LEVEL, as_index=False)
# #     .head(1)
# # )

# # STEP += 1
# # rep_arch.to_csv(
# #     OUTDIR / f"[STEP:{STEP}]representative_architectures.csv",
# #     index=False
# # )

# # logging.info(f"Selected {len(rep_arch)} representative architectures")

# # # ------------------------------------------------------------------------------
# # # Step 7: Recover representative contigs
# # # ------------------------------------------------------------------------------
# # logging.info("Recovering representative contigs")

# # rep_keys = rep_arch.merge(
# #     arch_df,
# #     on=[LINEAGE_LEVEL, "arch_token_canonical"],
# #     how="left"
# # )


# # STEP += 1
# # rep_keys.to_csv(
# #     OUTDIR / f"[STEP:{STEP}]representative_contigs.csv",
# #     index=False
# # )

# # # ------------------------------------------------------------------------------
# # # Step 8: Recover gene-level rows for plotting
# # # ------------------------------------------------------------------------------
# # logging.info("Extracting gene-level rows for plotting")

# # plot_df = df.merge(
# #     rep_keys[["genome_id_x", "contig", "center_protein"]],
# #     on=["genome_id_x", "contig", "center_protein"],
# #     how="inner"
# # )

# # plot_df = plot_df.sort_values(
# #     [LINEAGE_LEVEL, "genome_id_x", "contig", "relative_pos"]
# # )

# # STEP += 1

# # plot_df.to_csv(
# #     OUTDIR / f"[STEP:{STEP}]plot_ready_architectures.csv",
# #     index=False
# # )

# # ------------------------------------------------------------------------------
# # Summary
# # ------------------------------------------------------------------------------
# logging.info("Architecture analysis complete")
# logging.info(f"Results written to: {OUTDIR.resolve()}")

# logging.info("Files generated:")
# for f in sorted(OUTDIR.iterdir()):
#     logging.info(f"  - {f.name}")


In [55]:
# # Without Subtuple matching 


# ""#!/usr/bin/env python3
# """
# Architecture analysis and reduction script (orientation-invariant)
# =================================================================

# Input:
#   - Annotated neighborhood CSV with taxonomy and architecture labels

# Output:
#   - Contig-level trimmed & canonical architectures
#   - Lineage-level architecture counts (inverse-collapsed)
#   - Representative contigs per lineage
#   - Gene-level dataframe for plotting

# Key features:
#   - Flank trimming (k=2)
#   - Orientation-invariant (inverse) collapsing
#   - Counts incorporate reversed architectures
# """

# import pandas as pd
# import logging
# from pathlib import Path
# from datetime import datetime

# INPUT_CSV = out_csv # Output from previous step with taxonomy [should be [step:8]escrt_neighborhoods_with_taxonomy_{rightnow}.csv]
# OUTDIR = Path(os.path.join(MAIN_OUTDIR, f"architecture_results_{rightnow}"))


# OUTDIR.mkdir(exist_ok=True)


# STEP += 100
# # ------------------------------------------------------------------------------
# # Helper functions
# # ------------------------------------------------------------------------------
# def trim_architecture(tokens, k=2):
#     """
#     Trim architecture by keeping at most k 'other' tokens
#     at each flank while retaining internal structure.
#     """
#     tokens = list(tokens)

#     # Trim left
#     left = 0
#     while left < len(tokens) and tokens[left] == "other":
#         left += 1
#     left = max(0, left - k)

#     # Trim right
#     right = len(tokens)
#     while right > 0 and tokens[right - 1] == "other":
#         right -= 1
#     right = min(len(tokens), right + k)

#     return tuple(tokens[left:right])


# def canonicalize_architecture(tokens):
#     """
#     Make architecture orientation-invariant by collapsing
#     forward and reverse into a single canonical form.
#     """
#     tokens = tuple(tokens)
#     rev = tokens[::-1]
#     return min(tokens, rev)

# # ------------------------------------------------------------------------------
# # Step 1: Load data
# # ------------------------------------------------------------------------------
# logging.info("Loading annotated neighborhood table")
# df = pd.read_csv(INPUT_CSV)


# df['neighbor_architecture'] = df['neighbor_architecture'].str.strip()
# required_cols = {
#     "genome_id_x", "contig", "center_protein",
#     "relative_pos", "neighbor_architecture",
#     LINEAGE_LEVEL
# }

# missing = required_cols - set(df.columns)
# if missing:
#     raise ValueError(f"Missing required columns: {missing}")

# logging.info(f"Loaded {len(df)} gene-level rows")

# # ------------------------------------------------------------------------------
# # Step 2: Encode architecture tokens
# # ------------------------------------------------------------------------------
# logging.info("Encoding architecture tokens")

# df["arch_token"] = (
#     df["neighbor_architecture"]
#     .fillna("other")
# )

# # ------------------------------------------------------------------------------
# # Step 3: Build contig-level architectures
# # ------------------------------------------------------------------------------
# logging.info("Constructing contig-level architectures")

# arch_df = (
#     df
#     .sort_values("relative_pos")
#     .groupby(
#         ["genome_id_x", "contig", "center_protein", LINEAGE_LEVEL],
#         as_index=False
#     )
#     .agg(
#         arch_token=("arch_token", lambda x: tuple(x)),
#         n_genes=("arch_token", "count")
#     )
# )

# logging.info(f"Identified {len(arch_df)} contig-level architectures")

# STEP += 1
# arch_df.to_csv(OUTDIR / f"[STEP:{STEP}]contig_architectures_raw.csv", index=False)

# # ------------------------------------------------------------------------------
# # Step 4: Trim flanks + inverse collapse (CORE STEP)
# # ------------------------------------------------------------------------------
# logging.info(
#     f"Trimming architectures (k={FLANK_K}) and collapsing inverses"
# )

# arch_df["arch_token_trimmed"] = arch_df["arch_token"].apply(
#     lambda x: trim_architecture(x, k=FLANK_K)
# )

# arch_df["arch_token_canonical"] = arch_df["arch_token_trimmed"].apply(
#     canonicalize_architecture
# )

# arch_df["trimmed_len"] = arch_df["arch_token_trimmed"].apply(len)


# STEP += 1
# arch_df.to_csv(
#     OUTDIR / f"[STEP:{STEP}]contig_architectures_trimmed_canonical.csv",
#     index=False
# )

# # ------------------------------------------------------------------------------
# # Step 5: Count architectures by lineage (INVERSE-AWARE)
# # ------------------------------------------------------------------------------
# logging.info(
#     f"Counting architectures per {LINEAGE_LEVEL} (inverse-aware)"
# )

# arch_counts = (
#     arch_df
#     .groupby([LINEAGE_LEVEL, "arch_token_canonical"], as_index=False)
#     .size()
#     .rename(columns={"size": "count"})
#     .sort_values([LINEAGE_LEVEL, "count"], ascending=[True, False])
# )

# STEP += 1

# arch_counts.to_csv(
#     OUTDIR / f"[STEP:{STEP}]architecture_frequencies_inverse_collapsed.csv",
#     index=False
# )

# # # ------------------------------------------------------------------------------
# # # Step 6: Select representative architecture per lineage
# # # ------------------------------------------------------------------------------
# # logging.info("Selecting representative architecture per lineage")

# # rep_arch = (
# #     arch_counts
# #     .groupby(LINEAGE_LEVEL, as_index=False)
# #     .head(1)
# # )

# # STEP += 1
# # rep_arch.to_csv(
# #     OUTDIR / f"[STEP:{STEP}]representative_architectures.csv",
# #     index=False
# # )

# # logging.info(f"Selected {len(rep_arch)} representative architectures")

# # # ------------------------------------------------------------------------------
# # # Step 7: Recover representative contigs
# # # ------------------------------------------------------------------------------
# # logging.info("Recovering representative contigs")

# # rep_keys = rep_arch.merge(
# #     arch_df,
# #     on=[LINEAGE_LEVEL, "arch_token_canonical"],
# #     how="left"
# # )


# # STEP += 1
# # rep_keys.to_csv(
# #     OUTDIR / f"[STEP:{STEP}]representative_contigs.csv",
# #     index=False
# # )

# # # ------------------------------------------------------------------------------
# # # Step 8: Recover gene-level rows for plotting
# # # ------------------------------------------------------------------------------
# # logging.info("Extracting gene-level rows for plotting")

# # plot_df = df.merge(
# #     rep_keys[["genome_id_x", "contig", "center_protein"]],
# #     on=["genome_id_x", "contig", "center_protein"],
# #     how="inner"
# # )

# # plot_df = plot_df.sort_values(
# #     [LINEAGE_LEVEL, "genome_id_x", "contig", "relative_pos"]
# # )

# # STEP += 1

# # plot_df.to_csv(
# #     OUTDIR / f"[STEP:{STEP}]plot_ready_architectures.csv",
# #     index=False
# # )

# # # ------------------------------------------------------------------------------
# # # Summary
# # # ------------------------------------------------------------------------------
# # logging.info("Architecture analysis complete")
# # logging.info(f"Results written to: {OUTDIR.resolve()}")

# # logging.info("Files generated:")
# # for f in sorted(OUTDIR.iterdir()):
# #     logging.info(f"  - {f.name}")


In [56]:
# adding script to the log file
import json

# ------------------------------------------------------------------
# Load the current notebook file (Jupyter notebooks are JSON)
# ------------------------------------------------------------------
with open(NOTEBOOK_PATH, "r", encoding="utf-8") as nb:
    notebook_json = json.load(nb)


# ------------------------------------------------------------------
# Remove execution outputs from code cells
#   - prevents massive logs
#   - avoids embedding binary blobs (plots, images)
#   - keeps only the executable logic + structure
# ------------------------------------------------------------------
def strip_outputs(nb):
    for cell in nb.get("cells", []):
        if cell.get("cell_type") == "code":
            cell["outputs"] = []
            cell["execution_count"] = None
    return nb

notebook_json = strip_outputs(notebook_json)


# ------------------------------------------------------------------
# Serialize notebook JSON to a formatted string
#   - indentation preserves readability in logs
#   - done once to avoid repeated JSON encoding
# ------------------------------------------------------------------
notebook_str = json.dumps(notebook_json, indent=2)


# ------------------------------------------------------------------
# Append notebook snapshot to the log file
#   - logged line-by-line to preserve formatting
#   - ensures compatibility with logging handlers
# ------------------------------------------------------------------
logging.info("=" * 80)
logging.info("NOTEBOOK JSON SNAPSHOT (END OF RUN)")
logging.info("=" * 80)

for line in notebook_str.splitlines():
    logging.info(line)


2026-01-21 11:08:43,273 [INFO] ================================================================================
2026-01-21 11:08:43,274 [INFO] NOTEBOOK JSON SNAPSHOT (END OF RUN)
2026-01-21 11:08:43,274 [INFO] ================================================================================
2026-01-21 11:08:43,275 [INFO] {
2026-01-21 11:08:43,275 [INFO]   "cells": [
2026-01-21 11:08:43,276 [INFO]     {
2026-01-21 11:08:43,276 [INFO]       "cell_type": "code",
2026-01-21 11:08:43,276 [INFO]       "execution_count": null,
2026-01-21 11:08:43,277 [INFO]       "id": "99fd6d19",
2026-01-21 11:08:43,277 [INFO]       "metadata": {},
2026-01-21 11:08:43,278 [INFO]       "outputs": [],
2026-01-21 11:08:43,278 [INFO]       "source": [
2026-01-21 11:08:43,278 [INFO]         "# Configuration\n",
2026-01-21 11:08:43,279 [INFO]         "\n",
2026-01-21 11:08:43,279 [INFO]         "import logging\n",
2026-01-21 11:08:43,280 [INFO]         "import sys\n",
2026-01-21 11:08:43,280 [INFO]         "import 

In [58]:
import os
import shutil
import logging
from pathlib import Path
import pandas as pd
from Bio import SeqIO
from Bio.SeqFeature import SeqFeature




def compress_others(tokens, threshold=10):
    """
    Compress long runs of 'other' into 'other[n]' if n > threshold.
    """
    compressed = []
    i = 0

    while i < len(tokens):
        if tokens[i] != "other":
            compressed.append(tokens[i])
            i += 1
            continue

        # count run
        j = i
        while j < len(tokens) and tokens[j] == "other":
            j += 1

        run_len = j - i

        if run_len > threshold:
            compressed.append(f"other[{run_len}]")
        else:
            compressed.extend(["other"] * run_len)

        i = j

    return compressed

def extract_architecture_full_contig(anchor_df, gff_df, window=5):
    """
    Extract genomic neighborhoods around ESCRT-related anchor genes.
    """
    keywords = CORE_TARGETS

    logging.info(f"Starting neighborhood extraction (±{window} genes)")
    logging.info(f"Initial anchors: {len(anchor_df)}")

    # Filter anchors by keywords
    filtered_anchors = anchor_df[
        anchor_df["architecture"]
        .astype(str)
        .str.lower()
        .str.contains("|".join(keywords))
    ].copy()

    logging.info(f"Anchors after ESCRT filter: {len(filtered_anchors)}")

    if filtered_anchors.empty:
        logging.error("No ESCRT-related anchors found")
        sys.exit(1)

    neighborhoods = []

    # Group by genome and contig
    for (genome, contig), anchors in filtered_anchors.groupby(
        ["genome_id", "contig"]
    ):
        anchor_indices = anchors["gene_index"].values
        
        # Define window boundaries
        start = anchor_indices.min() - window
        end = anchor_indices.max() + window

        # Extract all genes in the window from gff_df
        block = gff_df[
            (gff_df["genome_id"] == genome) &
            (gff_df["contig"] == contig) &
            (gff_df["gene_index"].between(start, end))
        ].copy()

        if block.empty:
            continue

        # Sort by gene position
        block = block.sort_values("gene_index").reset_index(drop=True)

        # Create a mapping of protein_id to architecture from ALL anchors
        # (not just filtered ones), so we get complete architecture info
        arch_map = anchor_df.set_index("protein_id")["architecture"].to_dict()
        
        # Apply architecture: use anchor architecture if available, else "other"
        block["architecture"] = block["protein_id"].map(arch_map).fillna("other")

        # Build architecture token list for the ENTIRE neighborhood
        tokens = block["architecture"].tolist()
        compressed_tokens = compress_others(tokens, threshold=10)


        # Add metadata about the neighborhood
        block["num_anchors"] = len(anchors)

        block["window_start"] = start
        block["window_end"] = end
        block["neighborhood_architecture_compressed"] = ",".join(compressed_tokens)
        block["position_in_neighborhood"] = range(len(block))

        neighborhoods.append(block)

    if not neighborhoods:
        logging.error("No neighborhoods extracted after processing")
        sys.exit(1)

    combined = pd.concat(neighborhoods, ignore_index=True)

    logging.info(
        f"Extracted {len(combined)} genes from "
        f"{len(neighborhoods)} contig neighborhoods"
    )

    return combined

# -----------------------------------------------



# merge rep_genomes with anchor_df on genome_id
print(anchor_df.columns)
print(rep_genomes.columns)
rep_genomes_anchor = (
    rep_genomes
    .merge(
        anchor_df.drop_duplicates(),
        left_on=["genome_id_x", "contig"],
        right_on=["genome_id", "contig"],
        how="inner"
    )
)



logging.info(
    "Representative genomes retained: %d",
    rep_genomes_anchor["genome_id_x"].nunique()
)




# Extract full contig architecture to modify gbk files


gff_dir = Path("/home/anirudh/genomes/selected_genomes/prokka_results")


# --------------------------------------------------------
# Load GFFs
# --------------------------------------------------------
logging.info("\n[STEP 1] Loading GFF files...")
gff_df = load_gffs_from_hits(rep_genomes_anchor, gff_dir)



rep_df = extract_architecture_full_contig(rep_genomes_anchor, gff_df, window=1000)

rep_df.to_csv(
    OUTDIR / f"[STEP:{STEP}]representative_genomes_full_contig_architectures.csv",
    index=False
)

print("df:", df.columns)

# taxonomy_df must have genome_id and family
taxonomy_df = df[["genome_id_x", "family"]].drop_duplicates()

rep_df = rep_df.merge(
    taxonomy_df,
    left_on="genome_id",
    right_on="genome_id_x",
    how="left"
)

# sanity check
if rep_df["family"].isna().any():
    raise RuntimeError("Some rows in rep_df are missing family taxonomy")


# yield a csv file for gene function annotation in clinker protein_id, protein_name
print("rep_df columns:", rep_df.head(10))



annotation_df = rep_df[["protein_id", "architecture"]].drop_duplicates()

annotation_df = annotation_df[annotation_df["architecture"] != "other"]

annotation_csv =  OUTDIR / f"[STEP:{STEP}.9]representative_genomes_gene_function_annotation.csv"

annotation_df.to_csv(
    annotation_csv,
    index=False
)


Index(['target', 'Name', 'Completeness', 'Contamination', 'Contig_N50',
       'Total_Contigs', 'organism_name', 'architecture', 'architecture_method',
       'architecture_components', 'genome_id', 'contig', 'gene_index',
       'protein_id', 'start', 'end', 'strand'],
      dtype='object')
Index(['family', 'genome_id_x', 'rep_arch_canonical', 'contig'], dtype='object')
2026-01-21 11:11:06,024 [INFO] Representative genomes retained: 16
2026-01-21 11:11:06,025 [INFO] 
[STEP 1] Loading GFF files...
2026-01-21 11:11:06,026 [INFO] Loading GFFs for 16 unique genomes
2026-01-21 11:11:06,026 [INFO] Parsing GFF: GCA_001940665.2_ASM194066v2_genomic.gff
2026-01-21 11:11:06,031 [INFO]   → Parsed 1464 CDS features across 1 contigs
2026-01-21 11:11:06,034 [INFO] Parsing GFF: GCA_030668875.1_ASM3066887v1_genomic.gff
2026-01-21 11:11:06,046 [INFO]   → Parsed 2972 CDS features across 99 contigs
2026-01-21 11:11:06,048 [INFO] Parsing GFF: GCA_016840465.1_ASM1684046v1_genomic.gff
2026-01-21 11:11:06,05

In [60]:
from pathlib import Path


MAIN_OUTDIR = Path(MAIN_OUTDIR)
GBK_OUT_DIR = MAIN_OUTDIR / f"gbk_family_organized_{rightnow}"
GBK_OUT_DIR.mkdir(exist_ok=True)
gbk_out = MAIN_OUTDIR / "GBK_patched_dir"
gbk_out.mkdir(exist_ok=True)


# Copy GBK files organized by family
for family, fam_df in rep_df.groupby("family"):
    fam_dir = GBK_OUT_DIR / family
    fam_dir.mkdir(exist_ok=True)

    for genome in fam_df["genome_id"].unique():  # Fixed: removed _x suffix
        src_gbk = gff_dir / f"{genome}_genomic" / f"{genome}_genomic.gbk"
        dst_gbk = fam_dir / f"{genome}.gbk"

        if not src_gbk.exists():
            logging.warning("Missing GBK: %s", src_gbk)
            continue

        shutil.copy(src_gbk, dst_gbk)
        logging.info(f"Copied {genome}.gbk to {family}/")

logging.info("GBK files copied into family directories")


2026-01-21 11:11:15,909 [INFO] Copied GCA_016840465.1_ASM1684046v1.gbk to Baldrarchaeaceae/
2026-01-21 11:11:15,912 [INFO] Copied GCA_019056805.1_ASM1905680v1.gbk to DXJG01/
2026-01-21 11:11:15,916 [INFO] Copied GCA_038820475.1_ASM3882047v1.gbk to Freyrarchaeaceae/
2026-01-21 11:11:15,921 [INFO] Copied GCA_005191425.1_ASM519142v1.gbk to HEL-GB-B/
2026-01-21 11:11:15,925 [INFO] Copied GCA_030668875.1_ASM3066887v1.gbk to Heimdallarchaeaceae/
2026-01-21 11:11:15,929 [INFO] Copied GCA_003144275.1_ASM314427v1.gbk to Hodarchaeaceae/
2026-01-21 11:11:15,933 [INFO] Copied GCA_030614005.1_ASM3061400v1.gbk to JABLTI01/
2026-01-21 11:11:15,937 [INFO] Copied GCA_029856435.1_ASM2985643v1.gbk to Jordiarchaeaceae/
2026-01-21 11:11:15,940 [INFO] Copied GCA_030149205.1_ASM3014920v1.gbk to Kariarchaeaceae/
2026-01-21 11:11:15,944 [INFO] Copied GCA_021498095.1_ASM2149809v1.gbk to MK-D1/
2026-01-21 11:11:15,947 [INFO] Copied GCA_026993975.1_ASM2699397v1.gbk to Njordarchaeaceae/
2026-01-21 11:11:15,950 [IN

In [61]:
# from Bio import SeqIO
# from tqdm import tqdm
# import logging

# def has_gene(record):
#     """Check if a record has any CDS or gene features."""
#     return any(
#         f.type in {"CDS", "gene"} 
#         for f in record.features
#     )

# def patch_gbk_with_architecture(
#     gbk_path,
#     arch_df,
#     protein_col="protein_id",
#     arch_col="architecture",
#     rep_col="rep_arch_canonical"
# ):
#     """
#     Patch GenBank file with architecture annotations.
#     Modifies CDS features to include architecture information.
#     Filters out empty contigs after patching.
#     """
#     print(arch_df.columns)
#     # Create lookup dictionary
#     lookup = arch_df.set_index(protein_col).to_dict("index")
#     logging.info(f"Lookup keys sample: {list(lookup.keys())[:5]}")
    
#     logging.info(f"Patching {gbk_path.name} with {len(lookup)} annotated proteins")

#     records = []
#     total_genes_found = 0
#     total_genes_annotated = 0

#     for record in SeqIO.parse(gbk_path, "genbank"):
#         new_features = []
#         genes_found = 0
#         genes_annotated = 0

#         for feat in record.features:
#             # Keep non-CDS features as-is
#             if feat.type != "CDS":
#                 new_features.append(feat)
#                 continue

#             genes_found += 1
            
#             # Get protein_id from the CDS feature
#             pid = feat.qualifiers.get("locus_tag", [None])[0]

#             if pid is None or pid not in lookup:
#                 # Gene is outside annotation windows - skip
#                 continue

#             # Get architecture info
#             info = lookup[pid]
#             arch = info.get(arch_col, "other")
#             rep = info.get(rep_col, "NA")

#             genes_annotated += 1

#             # Modify the CDS feature qualifiers
#             if arch != "other":
#                 feat.qualifiers["gene"] = [arch]
#                 feat.qualifiers["product"] = [arch]
#             else:
#                 continue
            
#             # Add architecture notes
#             if "note" not in feat.qualifiers:
#                 feat.qualifiers["note"] = []
            
#             feat.qualifiers["note"].append(f"architecture={arch}")
#             if rep != "NA":
#                 feat.qualifiers["note"].append(f"representative_arch={rep}")
            
#             new_features.append(feat)

#         record.features = new_features
#         total_genes_found += genes_found
#         total_genes_annotated += genes_annotated
        
#         # Only keep records that have genes after patching
#         if has_gene(record):
#             records.append(record)
#             logging.info(
#                 f"  {record.id}: {genes_annotated}/{genes_found} genes annotated - KEPT"
#             )
#         else:
#             logging.info(
#                 f"  {record.id}: {genes_annotated}/{genes_found} genes annotated - REMOVED (empty)"
#             )

#     gbk_file = gbk_out / f"{arch_df['family'].unique()[0]}_patched.gbk"
#     print(gbk_file)
    
#     out_gbk_path = gbk_file
    
#     # Write modified records back
#     with open(out_gbk_path, "w") as out_handle:
#         SeqIO.write(records, out_handle, "genbank")
    
#     logging.info(
#         f"Successfully patched {gbk_path.name}: "
#         f"kept {len(records)} contigs, "
#         f"annotated {total_genes_annotated}/{total_genes_found} genes"
#     )
    
#     return len(records), total_genes_annotated


# # Patch all GBK files with architecture annotations
# for family, fam_df in tqdm(rep_df.groupby("family")):
#     fam_dir = GBK_OUT_DIR / family
    
#     logging.info(f"Patching family: {family} ({len(fam_df)} annotations)")

#     for gbk_file in fam_dir.glob("*.gbk"):
#         # Extract genome_id from filename
#         genome_id = gbk_file.stem  # removes .gbk extension
        
#         # Filter annotations for this specific genome
#         genome_df = fam_df[fam_df["genome_id"] == genome_id]
        
#         if genome_df.empty:
#             logging.warning(f"No annotations found for {genome_id}, skipping")
#             continue
        
#         num_contigs, num_genes = patch_gbk_with_architecture(
#             gbk_path=gbk_file,
#             arch_df=genome_df
#         )

# logging.info("All GBK files patched with architecture annotations and filtered")

In [62]:
from Bio import SeqIO
from Bio.SeqFeature import SeqFeature, FeatureLocation
from Bio.Seq import Seq
from tqdm import tqdm
import logging
import time
import random
# ============================================================================
# CONFIGURATION PARAMETERS
# ============================================================================
UPSTREAM_BUFFER = 1000      # Max bp to keep upstream of first gene
GAP_THRESHOLD = 10000       # Gap size that triggers truncation (25kb)
GAP_BUFFER = 1000           # bp to keep on each side of truncated gap
MIN_CONTIG_SIZE = 0         # Minimum contig size after truncation (0 = no limit)


# ============================================================================


class GapStringGenerator:
    """Generates gap strings with minimal similarity to existing gaps"""
    
    def __init__(self, gap_size, num_candidates=500):
        """
        Args:
            gap_size: Length of each gap string
            num_candidates: Number of candidates to evaluate per generation
        """
        self.gap_size = gap_size
        self.num_candidates = num_candidates
        self.existing_gaps = []
        self.nucleotides = ['A', 'C', 'G', 'T']
    
    def get_next_gap(self):
        """Generate and return the next gap string"""
        if len(self.existing_gaps) == 0:
            # First gap is random
            gap = self._random_gap()
        else:
            # Find gap with minimal similarity to existing gaps
            gap = self._find_dissimilar_gap()
        
        self.existing_gaps.append(gap)
        return gap
    
    def _random_gap(self):
        """Generate a random nucleotide string"""
        return ''.join(random.choices(self.nucleotides, k=self.gap_size))
    
    def _find_dissimilar_gap(self):
        """Find the most dissimilar gap from candidates"""
        best_gap = None
        best_min_dissimilarity = -1
        
        for _ in range(self.num_candidates):
            candidate = self._random_gap()
            
            # Find minimum dissimilarity to any existing gap
            min_dissimilarity = min(
                self._hamming_distance(candidate, existing)
                for existing in self.existing_gaps
            )
            
            # Keep candidate with highest minimum dissimilarity
            if min_dissimilarity > best_min_dissimilarity:
                best_min_dissimilarity = min_dissimilarity
                best_gap = candidate
        
        return best_gap
    
    def _hamming_distance(self, s1, s2):
        """Calculate number of differing positions"""
        return sum(c1 != c2 for c1, c2 in zip(s1, s2))
    
    def reset(self):
        """Clear all existing gaps and start fresh"""
        self.existing_gaps = []
    
    def get_gap_count(self):
        """Return number of gaps generated so far"""
        return len(self.existing_gaps)


GAP_FEAT_SIZE = 100  # Symbolic gap feature size
gap_string_generator = GapStringGenerator(gap_size=GAP_FEAT_SIZE)


def has_gene(record):
    """Check if a record has any CDS or gene features."""
    return any(f.type in {"CDS", "gene"} for f in record.features)

def patch_gbk_with_architecture(
    gbk_path,
    arch_df,
    gbk_out,
    protein_col="protein_id",
    arch_col="architecture",
    rep_col="rep_arch_canonical"
):
    """
    STEP 1: Patch GenBank file with architecture annotations.
    Modifies CDS features to include architecture information.
    Filters out genes not in lookup and genes with "other" architecture.
    """
    print(arch_df.columns)
    # Create lookup dictionary
    lookup = arch_df.set_index(protein_col).to_dict("index")
    logging.info(f"Lookup keys sample: {list(lookup.keys())[:5]}")
    
    logging.info(f"Patching {gbk_path.name} with {len(lookup)} annotated proteins")

    records = []
    total_genes_found = 0
    total_genes_annotated = 0

    for record in SeqIO.parse(gbk_path, "genbank"):
        new_features = []
        genes_found = 0
        genes_annotated = 0

        for feat in record.features:
            # Keep non-CDS features as-is
            if feat.type != "CDS":
                new_features.append(feat)
                continue

            genes_found += 1
            
            # Get protein_id from the CDS feature
            pid = feat.qualifiers.get("locus_tag", [None])[0]

            if pid is None or pid not in lookup:
                # Gene is outside annotation windows - DISCARD
                continue

            # Get architecture info
            info = lookup[pid]
            arch = info.get(arch_col, "other")
            rep = info.get(rep_col, "NA")

            # Only keep genes with real architecture (discard "other")
            if arch == "other":
                continue

            genes_annotated += 1

            # Modify the CDS feature qualifiers for clinker compatibility
            feat.qualifiers["gene"] = [arch]
            feat.qualifiers["product"] = [arch]  # Clinker uses this for labeling
            
            # Add architecture notes
            if "note" not in feat.qualifiers:
                feat.qualifiers["note"] = []
            
            feat.qualifiers["note"].append(f"architecture={arch}")
            if rep != "NA":
                feat.qualifiers["note"].append(f"representative_arch={rep}")
            
            new_features.append(feat)

        record.features = new_features
        total_genes_found += genes_found
        total_genes_annotated += genes_annotated
        
        # Only keep records that have genes after patching
        if has_gene(record):
            records.append(record)
            logging.info(
                f"  {record.id}: {genes_annotated}/{genes_found} genes annotated - KEPT"
            )
        else:
            logging.info(
                f"  {record.id}: {genes_annotated}/{genes_found} genes annotated - REMOVED (empty)"
            )

    gbk_file = gbk_out / f"{arch_df['family'].unique()[0]}_patched.gbk"
    print(gbk_file)
    
    out_gbk_path = gbk_file
    
    # Write modified records back
    with open(out_gbk_path, "w") as out_handle:
        SeqIO.write(records, out_handle, "genbank")
    
    logging.info(
        f"Successfully patched {gbk_path.name}: "
        f"kept {len(records)} contigs, "
        f"annotated {total_genes_annotated}/{total_genes_found} genes"
    )
    
    return out_gbk_path, len(records), total_genes_annotated


def create_gap_feature(start, end, gap_size, gap_id):
    """Create a CDS feature representing a truncated gap."""
    gap_label = f"gap [{gap_size:,} bp]"
    gap_feature = SeqFeature(
        FeatureLocation(start, end),
        type="CDS",
        qualifiers={
            "gene": [gap_label],
            "product": [gap_label],
            "note": [f"Truncated gap: {gap_size:,} bp removed"],
            "pseudo": ["true"]
        }
    )
    return gap_feature


def truncate_large_gaps(
    gbk_path,
    gbk_out,
    gap_generator,
    upstream_buffer=UPSTREAM_BUFFER,
    gap_threshold=GAP_THRESHOLD,
    gap_buffer=GAP_BUFFER,
    min_contig_size=MIN_CONTIG_SIZE
):
    """
    STEP 2: Truncate large gaps in already-filtered GenBank files.
    
    Args:
        gbk_path: Path to the filtered/patched GenBank file
        gbk_out: Output directory
        upstream_buffer: Max bp to keep upstream of first gene
        gap_threshold: Gap size that triggers truncation
        gap_buffer: bp to keep on each side of a truncated gap
        min_contig_size: Minimum contig size after truncation (0 = no limit)
    """
    logging.info(f"Truncating large gaps in {gbk_path.name}")
    
    # Create truncated subdirectory
    truncated_dir = gbk_out / "truncated"
    truncated_dir.mkdir(exist_ok=True)

    
    records = []
    
    for record in SeqIO.parse(gbk_path, "genbank"):
        # Get only CDS features
        cds_features = [f for f in record.features if f.type == "CDS"]
        non_cds_features = [f for f in record.features if f.type != "CDS"]
        
        if not cds_features:
            records.append(record)
            continue
        
        # Sort CDS features by start position
        sorted_cds = sorted(cds_features, key=lambda f: f.location.start)
        
        # Calculate truncation map and build new sequence
        position_map = {}
        new_pos = 0
        new_features = []
        new_seq_parts = []
        
        # Handle upstream region before first gene
        first_gene_start = sorted_cds[0].location.start
        upstream_start = max(0, first_gene_start - upstream_buffer)
        
        # Add upstream sequence
        if upstream_start < first_gene_start:
            new_seq_parts.append(record.seq[upstream_start:first_gene_start])
            for i in range(upstream_start, first_gene_start):
                position_map[i] = new_pos
                new_pos += 1
        
        # Process each gene and gaps between them
        for i, feat in enumerate(sorted_cds):
            gene_start = feat.location.start
            gene_end = feat.location.end
            
            # Map gene positions
            for pos in range(gene_start, gene_end):
                position_map[pos] = new_pos
                new_pos += 1
            
            # Add gene sequence
            new_seq_parts.append(record.seq[gene_start:gene_end])
            
            # Update feature location
            new_feat = SeqFeature(
                FeatureLocation(position_map[gene_start], new_pos, strand=feat.location.strand),
                type=feat.type,
                qualifiers=feat.qualifiers.copy()
            )
            new_features.append(new_feat)
            
            # Handle gap to next gene
            if i < len(sorted_cds) - 1:
                next_gene_start = sorted_cds[i + 1].location.start
                gap_size = next_gene_start - gene_end
                
                if gap_size > gap_threshold:
                    # Large gap - truncate it
                    # Keep gap_buffer on current gene side
                    buffer_end = gene_end + gap_buffer
                    for pos in range(gene_end, buffer_end):
                        position_map[pos] = new_pos
                        new_pos += 1
                    new_seq_parts.append(record.seq[gene_end:buffer_end])
                    
                    # Add gap feature
                    gap_feat_start = new_pos
                    gap_feat_size = gap_generator.gap_size
                    gap_feat = create_gap_feature(gap_feat_start, gap_feat_start + gap_feat_size, gap_size - 2 * gap_buffer, None)
                    new_features.append(gap_feat)
                    
                    # Add placeholder sequence for gap feature (Ns)
                    new_seq_parts.append(Seq(gap_generator.get_next_gap()))
                    new_pos += gap_feat_size
                    
                    # Keep gap_buffer on next gene side
                    buffer_start = next_gene_start - gap_buffer
                    for pos in range(buffer_start, next_gene_start):
                        position_map[pos] = new_pos
                        new_pos += 1
                    new_seq_parts.append(record.seq[buffer_start:next_gene_start])
                else:
                    # Small gap - keep it all
                    for pos in range(gene_end, next_gene_start):
                        position_map[pos] = new_pos
                        new_pos += 1
                    new_seq_parts.append(record.seq[gene_end:next_gene_start])
        
        # Create new record
        new_record = record[:]
        new_record.seq = sum(new_seq_parts, Seq(""))
        new_record.features = new_features
        
        # Check minimum contig size
        if min_contig_size > 0 and len(new_record.seq) < min_contig_size:
            logging.info(
                f"  {record.id}: {len(record.seq):,} bp -> {len(new_record.seq):,} bp "
                f"- DISCARDED (below minimum size {min_contig_size:,} bp)"
            )
            continue
        
        logging.info(
            f"  {record.id}: {len(record.seq):,} bp -> {len(new_record.seq):,} bp "
            f"({len(record.seq) - len(new_record.seq):,} bp removed)"
        )
        
        records.append(new_record)
    
    # Output file in truncated subdirectory
    out_path = truncated_dir / gbk_path.name.replace("_patched.gbk", "_truncated.gbk")
    
    with open(out_path, "w") as out_handle:
        SeqIO.write(records, out_handle, "genbank")
    
    logging.info(f"Truncated gaps written to {out_path}")
    return out_path


# STEP 1: Patch all GBK files with architecture annotations
for family, fam_df in tqdm(rep_df.groupby("family")):
    fam_dir = GBK_OUT_DIR / family
    
    logging.info(f"Patching family: {family} ({len(fam_df)} annotations)")

    for gbk_file in fam_dir.glob("*.gbk"):
        # Extract genome_id from filename
        genome_id = gbk_file.stem  # removes .gbk extension
        
        # Filter annotations for this specific genome
        genome_df = fam_df[fam_df["genome_id"] == genome_id]
        
        if genome_df.empty:
            logging.warning(f"No annotations found for {genome_id}, skipping")
            continue
        
        patched_path, num_contigs, num_genes = patch_gbk_with_architecture(
            gbk_path=gbk_file,
            arch_df=genome_df,
            gbk_out=gbk_out
        )

logging.info("All GBK files patched with architecture annotations")

# STEP 2: Truncate large gaps in patched files
logging.info("Starting gap truncation...")

for patched_file in gbk_out.glob("*_patched.gbk"):
    truncate_large_gaps(
        gbk_path=patched_file,
        gbk_out=gbk_out,
        gap_generator=gap_string_generator,
        upstream_buffer=UPSTREAM_BUFFER,
        gap_threshold=GAP_THRESHOLD,
        gap_buffer=GAP_BUFFER,
        min_contig_size=MIN_CONTIG_SIZE
    )

logging.info("All files processed: patched and truncated")

  0%|          | 0/16 [00:00<?, ?it/s]

2026-01-21 11:11:15,992 [INFO] Patching family: Baldrarchaeaceae (12 annotations)
Index(['genome_id', 'contig', 'gene_index', 'protein_id', 'start', 'end',
       'strand', 'architecture', 'num_anchors', 'window_start', 'window_end',
       'neighborhood_architecture_compressed', 'position_in_neighborhood',
       'genome_id_x', 'family'],
      dtype='object')
2026-01-21 11:11:15,994 [INFO] Lookup keys sample: ['JPJMNJDP_00791', 'JPJMNJDP_00792', 'JPJMNJDP_00793', 'JPJMNJDP_00794', 'JPJMNJDP_00795']
2026-01-21 11:11:15,994 [INFO] Patching GCA_016840465.1_ASM1684046v1.gbk with 12 annotated proteins
2026-01-21 11:11:15,997 [INFO]   JAEOSH010000005.1: 0/78 genes annotated - REMOVED (empty)
2026-01-21 11:11:15,997 [INFO]   JAEOSH010000087.1: 0/5 genes annotated - REMOVED (empty)
2026-01-21 11:11:15,998 [INFO]   JAEOSH010000053.1: 0/17 genes annotated - REMOVED (empty)
2026-01-21 11:11:15,999 [INFO]   JAEOSH010000054.1: 0/14 genes annotated - REMOVED (empty)
2026-01-21 11:11:16,000 [INFO] 

  6%|▋         | 1/16 [00:00<00:01,  8.24it/s]

2026-01-21 11:11:16,112 [INFO] Patching family: DXJG01 (333 annotations)
Index(['genome_id', 'contig', 'gene_index', 'protein_id', 'start', 'end',
       'strand', 'architecture', 'num_anchors', 'window_start', 'window_end',
       'neighborhood_architecture_compressed', 'position_in_neighborhood',
       'genome_id_x', 'family'],
      dtype='object')
2026-01-21 11:11:16,115 [INFO] Lookup keys sample: ['DBDKMGEL_01461', 'DBDKMGEL_01462', 'DBDKMGEL_01463', 'DBDKMGEL_01464', 'DBDKMGEL_01465']
2026-01-21 11:11:16,115 [INFO] Patching GCA_019056805.1_ASM1905680v1.gbk with 333 annotated proteins
2026-01-21 11:11:16,116 [INFO]   DXJG01000011.1: 0/14 genes annotated - REMOVED (empty)
2026-01-21 11:11:16,118 [INFO]   DXJG01000012.1: 0/27 genes annotated - REMOVED (empty)
2026-01-21 11:11:16,119 [INFO]   DXJG01000013.1: 0/35 genes annotated - REMOVED (empty)
2026-01-21 11:11:16,120 [INFO]   DXJG01000014.1: 0/27 genes annotated - REMOVED (empty)
2026-01-21 11:11:16,121 [INFO]   DXJG01000015.1: 0

 19%|█▉        | 3/16 [00:00<00:01, 11.81it/s]

...

 88%|████████▊ | 14/16 [00:01<00:00,  6.05it/s]

2026-01-21 11:11:17,968 [INFO] Patching family: Thorarchaeaceae (328 annotations)
Index(['genome_id', 'contig', 'gene_index', 'protein_id', 'start', 'end',
       'strand', 'architecture', 'num_anchors', 'window_start', 'window_end',
       'neighborhood_architecture_compressed', 'position_in_neighborhood',
       'genome_id_x', 'family'],
      dtype='object')
2026-01-21 11:11:17,971 [INFO] Lookup keys sample: ['CEGGCEJE_01685', 'CEGGCEJE_01686', 'CEGGCEJE_01687', 'CEGGCEJE_01688', 'CEGGCEJE_01689']
2026-01-21 11:11:17,971 [INFO] Patching GCA_029855935.1_ASM2985593v1.gbk with 328 annotated proteins
2026-01-21 11:11:17,973 [INFO]   JAGSHN010000013.1: 0/42 genes annotated - REMOVED (empty)
2026-01-21 11:11:17,975 [INFO]   JAGSHN010000003.1: 0/55 genes annotated - REMOVED (empty)
2026-01-21 11:11:17,976 [INFO]   JAGSHN010000008.1: 0/47 genes annotated - REMOVED (empty)
2026-01-21 11:11:17,981 [INFO]   JAGSHN010000027.1: 0/127 genes annotated - REMOVED (empty)
2026-01-21 11:11:17,982 [INF

100%|██████████| 16/16 [00:02<00:00,  7.06it/s]

2026-01-21 11:11:18,259 [INFO] All GBK files patched with architecture annotations
2026-01-21 11:11:18,259 [INFO] Starting gap truncation...
2026-01-21 11:11:18,260 [INFO] Truncating large gaps in Freyrarchaeaceae_patched.gbk
2026-01-21 11:11:18,266 [INFO]   JAWBET010000001.1: 545,876 bp -> 20,902 bp (524,974 bp removed)
2026-01-21 11:11:18,268 [INFO] Truncated gaps written to /home/anirudh/synteny/hmms/ESCRT_synteny_pipeline_output_family_2026-01-21-11-08-12/GBK_patched_dir/truncated/Freyrarchaeaceae_truncated.gbk
2026-01-21 11:11:18,268 [INFO] Truncating large gaps in Njordarchaeaceae_patched.gbk
2026-01-21 11:11:18,269 [INFO]   JALWRR010000107.1: 17,870 bp -> 6,609 bp (11,261 bp removed)
2026-01-21 11:11:18,270 [INFO] Truncated gaps written to /home/anirudh/synteny/hmms/ESCRT_synteny_pipeline_output_family_2026-01-21-11-08-12/GBK_patched_dir/truncated/Njordarchaeaceae_truncated.gbk
2026-01-21 11:11:18,271 [INFO] Truncating large gaps in Heimdallarchaeaceae_patched.gbk
2026-01-21 11:

2026-01-21 11:11:19,011 [INFO]   JAIZWK010000001.1: 4,361,485 bp -> 94,927 bp (4,266,558 bp removed)
2026-01-21 11:11:19,013 [INFO] Truncated gaps written to /home/anirudh/synteny/hmms/ESCRT_synteny_pipeline_output_family_2026-01-21-11-08-12/GBK_patched_dir/truncated/MK-D1_truncated.gbk
2026-01-21 11:11:19,014 [INFO] Truncating large gaps in Wukongarchaeaceae_patched.gbk
2026-01-21 11:11:19,015 [INFO]   JAEOSI010000007.1: 23,656 bp -> 4,953 bp (18,703 bp removed)
2026-01-21 11:11:19,016 [INFO] Truncated gaps written to /home/anirudh/synteny/hmms/ESCRT_synteny_pipeline_output_family_2026-01-21-11-08-12/GBK_patched_dir/truncated/Wukongarchaeaceae_truncated.gbk
2026-01-21 11:11:19,016 [INFO] Truncating large gaps in JABLTI01_patched.gbk
2026-01-21 11:11:19,017 [INFO]   JAUWKS010000014.1: 62,672 bp -> 5,029 bp (57,643 bp removed)
2026-01-21 11:11:19,018 [INFO] Truncated gaps written to /home/anirudh/synteny/hmms/ESCRT_synteny_pipeline_output_family_2026-01-21-11-08-12/GBK_patched_dir/trunc

In [67]:


code_block = f"conda activate clinker\nclinker {gbk_out / 'truncated'}/*.gbk -o {gbk_out / 'truncated'}/ESCRT_SYNTENY --plot {gbk_out / 'truncated'}/ESCRT_synteny.html -gf {annotation_csv}"

logging.info("Clinker command for synteny visualization:\n%s", code_block)

print("\n")

2026-01-21 11:14:00,344 [INFO] Clinker command for synteny visualization:
conda activate clinker
clinker /home/anirudh/synteny/hmms/ESCRT_synteny_pipeline_output_family_2026-01-21-11-08-12/GBK_patched_dir/truncated/*.gbk -o /home/anirudh/synteny/hmms/ESCRT_synteny_pipeline_output_family_2026-01-21-11-08-12/GBK_patched_dir/truncated/ESCRT_SYNTENY --plot /home/anirudh/synteny/hmms/ESCRT_synteny_pipeline_output_family_2026-01-21-11-08-12/GBK_patched_dir/truncated/ESCRT_synteny.html -gf /home/anirudh/synteny/hmms/ESCRT_synteny_pipeline_output_family_2026-01-21-11-08-12/architecture_results_2026-01-21-11-08-12/[STEP:17.9]representative_genomes_gene_function_annotation.csv


